<a href="https://colab.research.google.com/github/00000281892/DynaBiome/blob/main/Paper_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project Title

Anomaly Detection in Longitudinal Microbiome Data using LSTM Autoencoders and Ensemble Learning.

## Overview

This reserach project implements an anomaly detection framework using LSTM Autoencoders to identify anomalous patterns in longitudinal microbiome data. The reconstruction error from the autoencoder is then used as a feature for various downstream classification models (Logistic Regression, MLP, One-Class SVM, K-Nearest Neighbors, XGBoost, Random Forest) and ensemble learning techniques (Averaged Probabilities, Weighted Averaging, Stacking) to improve anomaly detection performance.

The project includes:
- Data loading and preprocessing for time-series analysis.
- Training of an LSTM Autoencoder on normal data sequences.
- Evaluation of the autoencoder using reconstruction error analysis and optimal thresholding.
- Training and evaluation of individual classifiers on reconstruction errors.
- Implementation and evaluation of ensemble learning methods.
- Statistical testing and bootstrap analysis to compare model performances.
- Visualization of results including reconstruction error distribution, ROC and PR curves, confusion matrices, and SHAP explanations.

## Table of Contents

1. [Setup](#setup)
2. [Data](#data)
3. [Methodology](#methodology)
4. [Results](#results)
5. [Interpretation](#interpretation)
6. [Ensemble Learning](#ensemble-learning)
7. [Statistical Analysis](#statistical-analysis)
8. [File Structure](#file-structure)
9. [Dependencies](#dependencies)
10. [Usage](#usage)
11. [License](#license)
12. [Contact](#contact)

<a name="setup"></a>
## setup

In [ ]:
!pip install tensorflow
!pip install shap # Install the shap library before importing it.
!pip install statsmodels

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, RepeatVector, TimeDistributed, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint

<a name="data"></a>
## Data

In [ ]:
# Load csv file with pandas
import pandas as pd
df = pd.read_csv('/content/asv_interpretability_dataset_modified.csv', dtype={'PatientID': str})

<a name=""></a>

<a name="methodology"></a>
## Methodology

In [ ]:
# --- Handle NeutrophilCount with '<0.1' values ---
def parse_neutrophil(value):
    try:
        return float(value)
    except:
        if isinstance(value, str) and "<" in value:
            threshold = float(value.replace("<", "").strip())
            return threshold / 2  # or use threshold itself
        return np.nan

df['NeutrophilCount'] = df['NeutrophilCount'].apply(parse_neutrophil)

In [ ]:
# One-hot encode stool consistency
df = pd.get_dummies(df, columns=['Consistency'])

In [ ]:
# Log transform Genus-relative abundances
df['RelativeAbundance'] = df['RelativeAbundance'].astype(float)
df['RelativeAbundance'] = np.log1p(df['RelativeAbundance'])

In [ ]:
# Newly added section for testing - ground truth label
def label_dysbiosis(row):
    is_temp_abnormal = row['MaxTemperature'] > 38.0
    is_neutro_low = row['NeutrophilCount'] < 500
    is_consistency_liquid = row.get('Consistency_liquid', 0) == 1
    return int(is_temp_abnormal and is_neutro_low and is_consistency_liquid)

In [ ]:
df['DysbiosisLabel'] = df.apply(label_dysbiosis, axis=1)

In [ ]:
# Keep metadata along with abundances
metadata_cols = ['PatientID', 'SampleID', 'DayRelativeToNearestHCT',
                 'MaxTemperature', 'NeutrophilCount'] + \
                [col for col in df.columns if col.startswith('Consistency_')] + \
                ['DysbiosisLabel']

In [ ]:
# Pivot genus into columns (wide format)
genus_pivot = df.pivot_table(index=['PatientID', 'SampleID', 'DayRelativeToNearestHCT'],
                              columns='Genus', values='RelativeAbundance', fill_value=0).reset_index()

In [ ]:
# Merge with metadata
metadata = df[metadata_cols].drop_duplicates(subset=['PatientID', 'SampleID', 'DayRelativeToNearestHCT'])
merged_df = pd.merge(genus_pivot, metadata, on=['PatientID', 'SampleID', 'DayRelativeToNearestHCT'], how='left')

In [ ]:
# Drop genus columns with zero variance
genus_cols = df['Genus'].unique().tolist()
variances = merged_df[genus_cols].var()
non_zero_var_cols = variances[variances > 1e-6].index.tolist()

In [ ]:
# Final feature set
feature_cols = non_zero_var_cols + ['MaxTemperature', 'NeutrophilCount'] + \
               [col for col in merged_df.columns if 'Consistency' in col]

In [ ]:
# ==================== STEP 2: Scaling ====================
scaler = MinMaxScaler()
merged_df[feature_cols] = scaler.fit_transform(merged_df[feature_cols])

In [ ]:
# ==================== STEP 3: Prepare Time Series ====================
# Sort & reshape by patient and day
merged_df = merged_df.sort_values(['PatientID', 'DayRelativeToNearestHCT'])

In [ ]:
# --- [1] Build sequences with labels ---
def build_sequences_with_labels(df, feature_cols, label_col='DysbiosisLabel', seq_len=14):
    X_sequences = []
    y_labels = []
    time_indices = []

    for pid, group in df.groupby('PatientID'):
        group = group.sort_values('DayRelativeToNearestHCT')
        values = group[feature_cols].values
        labels = group[label_col].values
        days = group['DayRelativeToNearestHCT'].values

        for i in range(len(values) - seq_len + 1):
            seq = values[i:i+seq_len]
            label_window = labels[i:i+seq_len]
            label = int(label_window.max())  # 1 if any point in window is dysbiotic
            time = days[i+seq_len-1]  # assign the last day of the window
            X_sequences.append(seq)
            y_labels.append(label)
            time_indices.append((pid, time))

    return np.array(X_sequences), np.array(y_labels), time_indices

In [ ]:
# --- [2] Define features ---
seq_len = 14
feature_cols = [col for col in merged_df.columns
                if col not in ['PatientID', 'SampleID', 'DayRelativeToNearestHCT',
                               'DysbiosisLabel', 'MaxTemperature', 'NeutrophilCount']
                and not col.startswith('Consistency_')]

# --- [3] Build sequences ---
X_seq, y_seq, time_idx = build_sequences_with_labels(merged_df, feature_cols, seq_len=seq_len)

In [ ]:
# --- [4] Create DataFrame for temporal splitting ---
split_df = pd.DataFrame(time_idx, columns=['PatientID', 'DayRelativeToNearestHCT'])
split_df['y'] = y_seq
split_df['X_idx'] = range(len(X_seq))

In [ ]:
# --- [5] Sort by patient and time ---
split_df = split_df.sort_values(['PatientID', 'DayRelativeToNearestHCT'])

In [ ]:
# --- [6] Determine split indices (70/15/15 by number of sequences) ---
n_total = len(split_df)
n_train = int(0.7 * n_total)
n_val = int(0.15 * n_total)

In [ ]:
train_idx = split_df.iloc[:n_train]['X_idx'].values
val_idx = split_df.iloc[n_train:n_train+n_val]['X_idx'].values
test_idx = split_df.iloc[n_train+n_val:]['X_idx'].values

In [ ]:
# --- [7] Final splits ---
X_train, y_train = X_seq[train_idx], y_seq[train_idx]
X_val, y_val = X_seq[val_idx], y_seq[val_idx]
X_test, y_test = X_seq[test_idx], y_seq[test_idx]

In [ ]:
print(f"Train: {X_train.shape}, Validation: {X_val.shape}, Test: {X_test.shape}")

In [ ]:
# Filter X_train to only include normal samples (where y_train is 0)
X_train_normal = X_train[y_train == 0]

In [ ]:
timesteps = X_train_normal.shape[1]
n_features = X_train_normal.shape[2]

print(f"Timesteps: {timesteps}, Features: {n_features}")

In [ ]:
# ==================== STEP 4: LSTM Autoencoder ====================
inputs = Input(shape=(timesteps, n_features))
encoded = LSTM(128, return_sequences=True)(inputs)
encoded = LSTM(64, return_sequences=False)(encoded)
bottleneck = RepeatVector(timesteps)(encoded)
decoded = LSTM(64, return_sequences=True)(bottleneck)
decoded = LSTM(128, return_sequences=True)(decoded)
outputs = TimeDistributed(Dense(n_features))(decoded)

autoencoder = Model(inputs, outputs)
autoencoder.compile(optimizer=Adam(1e-3), loss='mae')

In [ ]:
# Train model
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

# Save best model based on lowest val_loss
checkpoint_cb = ModelCheckpoint(
    filepath="best_model_Unsupervised.keras",
    monitor="val_loss",
    save_best_only=True,
    verbose=1
)

In [ ]:
history = autoencoder.fit(X_train_normal , X_train_normal , epochs=300, batch_size=32, validation_data=(X_val, X_val), callbacks=[early_stop, checkpoint_cb], verbose=1)

In [ ]:
# Load the best model
from tensorflow.keras.models import load_model
best_autoencoder = load_model('best_model_Unsupervised.keras')

<a name="results"></a>
## Results

# Reconstruction loss trends during training and validation

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Reconstruction Loss Trends', fontsize=16)
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Loss (MAE)', fontsize=12)
plt.legend(fontsize=10)
plt.grid(True, linestyle='--', alpha=0.6)
plt.tight_layout() # Adjust layout to prevent labels overlapping
plt.savefig('reconstruction_loss_trends.pdf', dpi=600, format='pdf')
plt.show()

# Reconstruction Errors on Validation Set

In [ ]:
# Assuming 'autoencoder_model' is your trained autoencoder
# Assuming 'X_val' is your validation data

# Predict the reconstruction of X_val using the trained autoencoder
X_val_pred = autoencoder.predict(X_val)

# Calculate the Mean Absolute Error (MAE) between the original X_val and its reconstruction
reconstruction_errors_val = np.mean(np.abs(X_val_pred - X_val), axis=(1, 2))

# reconstruction_errors_val now contains a single reconstruction error value for each sequence in X_val

threshold = 0.0003 # np.percentile(reconstruction_errors_val, 95)
# Newly Added on 11 May, 2025
# predicted_labels = (reconstruction_errors >= threshold).astype(int)

print(f"\nAnomaly detection threshold (95th percentile): {threshold:.6f}")

# Reconstruction Errors on Test Set

In [ ]:
X_test_pred = autoencoder.predict(X_test)
reconstruction_errors_test = np.mean(np.abs(X_test_pred - X_test), axis=(1, 2))
anomaly_predictions = reconstruction_errors_test > threshold  # threshold from training on validation set. Use the Threshold obtained via validation set
print(f"Number of flagged sequences: {anomaly_predictions.sum()} out of {len(anomaly_predictions)}")

In [ ]:
from sklearn.metrics import roc_auc_score, average_precision_score, roc_curve, precision_recall_curve, confusion_matrix
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

# --- 1. Find Optimal Threshold on the Validation Set ---

# Compute ROC Curve on Validation Set to get thresholds
fpr_val, tpr_val, thresholds_val = roc_curve(y_val, reconstruction_errors_val)

# Find the best threshold using Youden's index on Validation Set
optimal_idx_val = np.argmax(tpr_val - fpr_val)
optimal_threshold = thresholds_val[optimal_idx_val]

print(f"Optimal Threshold for anomaly detection (from Validation Set): {optimal_threshold:.4f}")

# --- 2. Evaluate Performance and Plot on the Test Set using the Optimal Threshold ---

# Classify Test Instances using the optimal threshold found on the validation set
anomaly_predictions_test = (reconstruction_errors_test > optimal_threshold).astype(int)

# Compute Evaluation Metrics on the Test Set
roc_auc_test = roc_auc_score(y_test, reconstruction_errors_test) # AUC is calculated from scores, not binary predictions
pr_auc_test = average_precision_score(y_test, reconstruction_errors_test) # PR AUC is calculated from scores

# You can also calculate metrics based on the binary predictions
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report

precision_test = precision_score(y_test, anomaly_predictions_test, zero_division=1)
recall_test = recall_score(y_test, anomaly_predictions_test, zero_division=1)
f1_test = f1_score(y_test, anomaly_predictions_test, zero_division=1)

print("\nEvaluation on Test Set using Optimal Threshold from Validation:")
print(f"Precision: {precision_test:.4f}")
print(f"Recall: {recall_test:.4f}")
print(f"F1-Score: {f1_test:.4f}")
print(f"ROC AUC: {roc_auc_test:.4f}")
print(f"PR AUC: {pr_auc_test:.4f}")
print("\nClassification Report on Test Set:")
print(classification_report(y_test, anomaly_predictions_test, zero_division=1))


# --- 3. Plot ROC Curve on the Test Set ---

# Compute ROC Curve on Test Set (for plotting the curve itself)
fpr_test, tpr_test, thresholds_test_plot = roc_curve(y_test, reconstruction_errors_test)

plt.figure(figsize=(12, 5))

# --- ROC ---
plt.subplot(1, 2, 1)
plt.plot(fpr_test, tpr_test, label=f'ROC AUC = {roc_auc_test:.3f}', color='blue')

# Find the point on the Test ROC curve corresponding to the optimal threshold from Validation
# This requires finding the index in thresholds_test_plot closest to optimal_threshold
closest_threshold_idx_test = np.argmin(np.abs(thresholds_test_plot - optimal_threshold))
plt.scatter(fpr_test[closest_threshold_idx_test], tpr_test[closest_threshold_idx_test],
            color='red', label=f'Applied Threshold ({optimal_threshold:.4f})', zorder=5) # zorder to ensure point is visible

plt.plot([0, 1], [0, 1], 'k--')  # Diagonal reference line
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve (Evaluated on Test Set)")
plt.legend()
plt.grid(True)

# Save the ROC plot
# Define a suitable output directory if not already defined
output_dir_evaluation_plots = "/content/drive/MyDrive/Final_Implementations/DynaBiome_Figures/Evaluation_Plots/"
import os
os.makedirs(output_dir_evaluation_plots, exist_ok=True)

plt.savefig(f"{output_dir_evaluation_plots}/ROC_Curve_TestSet_ValidatedThreshold.pdf", dpi=600, bbox_inches='tight')

# --- 4. Plot Precision-Recall Curve on the Test Set ---

# Compute PR Curve on Test Set (for plotting the curve itself)
precision_test_plot, recall_test_plot, thresholds_pr_test_plot = precision_recall_curve(y_test, reconstruction_errors_test)

plt.subplot(1, 2, 2) # Plotting PR curve in the second subplot
plt.plot(recall_test_plot, precision_test_plot, label=f'PR AUC = {pr_auc_test:.3f}', color='red')

# Find the point on the Test PR curve corresponding to the optimal threshold from Validation
# Similar to ROC, find the index in thresholds_pr_test_plot closest to optimal_threshold
# Note: precision_recall_curve thresholds are slightly different in number/values than roc_curve
closest_threshold_idx_pr_test = np.argmin(np.abs(thresholds_pr_test_plot - optimal_threshold))
# Need to handle the case where thresholds_pr_test_plot might be empty (e.g., all samples are the same)
if thresholds_pr_test_plot.size > 0:
     plt.scatter(recall_test_plot[closest_threshold_idx_pr_test], precision_test_plot[closest_threshold_idx_pr_test],
                 color='blue', label=f'Applied Threshold ({optimal_threshold:.4f})', zorder=5)

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve (Evaluated on Test Set)')
plt.legend()
plt.grid(True)

# Save the PR plot
plt.savefig(f"{output_dir_evaluation_plots}/PR_Curve_TestSet_ValidatedThreshold.pdf", dpi=600, bbox_inches='tight')


plt.tight_layout() # Adjust layout to prevent overlap
plt.show()

# --- 5. Plot Confusion Matrix on the Test Set ---
cm_test = confusion_matrix(y_test, anomaly_predictions_test)

plt.figure(figsize=(8, 6))
sns.heatmap(cm_test, annot=True, fmt="d", cmap="Blues",
            xticklabels=['Predicted Normal', 'Predicted Anomaly'],
            yticklabels=['Actual Normal', 'Actual Anomaly'])
plt.title('Confusion Matrix (Evaluated on Test Set)')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.savefig(f"{output_dir_evaluation_plots}/ConfusionMatrix_TestSet_ValidatedThreshold.pdf", dpi=600, bbox_inches='tight')
plt.show()

# Reconstruction Error Distribution using optimal threshold.

In [ ]:
# Plot Reconstruction Error Distribution
plt.figure(figsize=(10,4))
# Filter reconstruction errors for normal (0) and anomaly (1) based on actual y_test labels
reconstruction_errors_normal = reconstruction_errors_test[y_test == 0]
reconstruction_errors_anomaly = reconstruction_errors_test[y_test == 1]

# Plot histograms for both classes
plt.hist(reconstruction_errors_normal, bins=50, color='skyblue', edgecolor='black', alpha=0.7, label='Actual Normal')
plt.hist(reconstruction_errors_anomaly, bins=50, color='salmon', edgecolor='black', alpha=0.7, label='Actual Anomaly')

# Add the optimal threshold line from validation set
plt.axvline(optimal_threshold, color='red', linestyle='--', label=f'Optimal Threshold ({optimal_threshold:.4f})')

plt.title("Reconstruction Error Distribution by Actual Class (Evaluated on Test Set)")
plt.xlabel("Reconstruction Error (MAE)")
plt.ylabel("Frequency")
plt.legend()
plt.tight_layout()

# Save the distribution plot
plt.savefig(f"{output_dir_evaluation_plots}/reconstruction_error_distribution_by_class_TestSet_ValidatedThreshold.pdf", dpi=600, bbox_inches='tight')
plt.show()


<a name=""></a>
## Interpretation

In [ ]:
import shap
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Define a wrapper model to calculate reconstruction error (if not already defined and accessible)
# If ReconstructionErrorModel is defined in a previous cell and accessible, you can skip this part.
class ReconstructionErrorModel:
    def __init__(self, autoencoder): # Assuming autoencoder is available in the environment
        self.autoencoder = autoencoder

    def predict(self, X):
        # Reshape X to 3D before passing to autoencoder
        # Assuming timesteps and n_features are defined globally and accessible
        X_reshaped = X.reshape(-1, timesteps, n_features)

        # Calculate reconstruction errors
        X_pred = self.autoencoder.predict(X_reshaped)
        reconstruction_error = np.mean(np.abs(X_reshaped - X_pred), axis=(1, 2))  # Scalar per sample
        return reconstruction_error

# Initialize the wrapper model (assuming autoencoder is available in the environment)
# If 'autoencoder' and 'n_features' are not globally accessible, you need to ensure they are loaded/defined.
try:
    # This line assumes 'autoencoder' was loaded successfully in a previous cell
    reconstruction_model = ReconstructionErrorModel(autoencoder)
except NameError:
    print("Error: 'autoencoder' model not found. Please ensure it is loaded before this cell.")
    # You might need to load the model here if it's not guaranteed to be in the environment
    # from tensorflow.keras.models import load_model
    # autoencoder = load_model("Proposed_Model_Binary_FullUnsupervised.keras")
    # reconstruction_model = ReconstructionErrorModel(autoencoder)
    # You might also need to define n_features if it's not globally accessible
    # n_features = X_train_normal.shape[2] # Or get from model input shape

# Initialize SHAP explainer (Using a subset of training data as background data)
# Assuming X_train is available from previous steps
try:
    rng = np.random.default_rng(seed=42)
    # Use a subset for efficiency and reshape background data
    explainer = shap.KernelExplainer(reconstruction_model.predict, X_train[:100].reshape(X_train[:100].shape[0], -1), rng=rng)
except NameError:
    print("Error: 'X_train' or 'reconstruction_model' not found. Ensure previous steps ran successfully.")
    # Handle the error appropriately, perhaps by exiting or loading necessary data

In [ ]:
import shap
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# ----- Preprocess: Reshape and Get SHAP Values -----
# Only explain the first 10 test samples
X_test_reshape = X_test[:10].reshape(X_test[:10].shape[0], -1)
shap_values_test = explainer.shap_values(X_test_reshape)

# ----- Generate Feature Names Across Time Steps -----
flattened_feature_names = [
    f"{feature}_t{t}" for t in range(timesteps) for feature in feature_cols
]

In [ ]:
# ----- Set High-Quality Plotting Parameters -----
sns.set(style="whitegrid", font_scale=1.5)
plt.rcParams.update({
    "axes.titlesize": 20,
    "axes.labelsize": 16,
    "xtick.labelsize": 12,
    "ytick.labelsize": 12,
    "legend.fontsize": 12,
    "font.family": "serif"
})

# ----- Create SHAP Summary Plot with Matplotlib Backend -----
shap.summary_plot(
    shap_values_test,                        # SHAP values for the selected samples
    features=X_test_reshape,                 # Corresponding input features
    feature_names=flattened_feature_names,   # Custom time-based feature names
    max_display=30,                          # Top 30 features
    plot_type="dot",                         # Use dot plot
    color_bar=True,                          # Show feature value color bar
    plot_size=(12, 10),                      # Increase size for clarity
    show=False                              # Don't auto-show, allows customization
    # matplotlib=True                          # Use matplotlib backend for quality
)

# ----- Save Plot in High Resolution (PNG + Optional Vector Format) -----
plt.tight_layout()
plt.savefig("SHAP_Summary_Plot_HighRes.pdf", dpi=600, bbox_inches="tight")      # For raster
plt.savefig("SHAP_Summary_Plot_Vector.svg", bbox_inches="tight")               # For vector
plt.show()

In [ ]:
import shap
import matplotlib.pyplot as plt
import seaborn as sns

# ----- Set High-Quality Plotting Parameters -----
sns.set(style="whitegrid", font_scale=1.5)
plt.rcParams.update({
    "axes.titlesize": 20,
    "axes.labelsize": 16,
    "xtick.labelsize": 14,
    "ytick.labelsize": 14,
    "legend.fontsize": 12,
    "font.family": "serif"
})

# ----- Create SHAP Summary Plot -----
shap.summary_plot(
    shap_values_test,                        # SHAP values
    features=X_test_reshape,                 # Input features
    feature_names=flattened_feature_names,   # Custom feature names
    max_display=30,                          # Top 30 features
    plot_type="dot",                         # Dot plot
    color_bar=True,                          # Color bar enabled
    plot_size=(12, 10),                      # Figure size
    show=False                               # Defer final show
)

# ----- Post-processing: Add Title and Bold Axis Labels -----
plt.title("Feature Contribution to Model Output (SHAP Summary Plot)", fontsize=20, fontweight='bold')
plt.xlabel("SHAP Value (Impact on Model Output)", fontsize=16, fontweight='bold')
plt.ylabel("Top Contributing Features", fontsize=16, fontweight='bold')

# Bold tick labels
plt.xticks(fontsize=14, fontweight='bold')
plt.yticks(fontsize=14, fontweight='bold')

# ----- Save in High Resolution (Raster + Vector) -----
plt.tight_layout()
plt.savefig("SHAP_Summary_Plot_HighRes.pdf", dpi=600, bbox_inches="tight")    # Raster (PDF)
plt.savefig("SHAP_Summary_Plot_Vector.svg", bbox_inches="tight")             # Vector (SVG)

# ----- Final Display -----
plt.show()

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Set global plot style for publication
sns.set(style="whitegrid")
plt.rcParams.update({
    'font.family': 'serif',
    'axes.titlesize': 18,
    'axes.labelsize': 16,
    'xtick.labelsize': 13,
    'ytick.labelsize': 13,
    'figure.dpi': 300
})

# Reconstruct predictions
X_pred_test = autoencoder.predict(X_test)
reconstruction_errors = np.mean(np.abs(X_pred_test - X_test), axis=(1, 2))
feature_errors = np.abs(X_pred_test - X_test)
feature_errors_avg = feature_errors.mean(axis=1)

# Anomaly filtering
anomaly_flags = reconstruction_errors > optimal_threshold
anomalous_feature_errors = feature_errors_avg[anomaly_flags]
mean_error_per_feature = anomalous_feature_errors.mean(axis=0)

# DataFrame preparation
error_df = pd.DataFrame({
    'Feature': feature_cols,
    'MeanAnomalyError': mean_error_per_feature
}).sort_values(by='MeanAnomalyError', ascending=True)  # sort ascending for horizontal bars

# Plotting
plt.figure(figsize=(10, 8))
barplot = sns.barplot(
    data=error_df.tail(20),  # Top 20 highest
    x='MeanAnomalyError',
    y='Feature',
    palette='coolwarm'
)

# Annotate bars
for p in barplot.patches:
    width = p.get_width()
    barplot.text(
        width + 0.0005,  # Offset for readability
        p.get_y() + p.get_height() / 2,
        f'{width:.3f}',
        ha='left',
        va='center',
        fontsize=10
    )

# Titles and labels
plt.title("Top Contributing Genera/Features to Anomalous Patterns", fontsize=18, weight='bold')
plt.xlabel("Mean Absolute Error (Anomalous Sequences)", fontsize=14, weight='bold')
plt.ylabel("Genus / Feature", fontsize=14, weight='bold')

plt.tight_layout()

# Save as high-quality figure
plt.savefig("Top_Contributing_Genera.pdf", dpi=600, bbox_inches='tight')
# plt.savefig("Top_Contributing_Genera.svg", bbox_inches='tight')
plt.show()

## Correlation of Predicted Anomalies with Clinical Markers such as neutrophil counts, temperature, and stool consistency.

In [ ]:
# Load the best model
best_autoencoder = load_model('best_model_Unsupervised.keras')

# Calculate reconstruction error on all sets
train_preds_1 = best_autoencoder.predict(X_train)
val_preds_1 = best_autoencoder.predict(X_val)
test_preds_1 = best_autoencoder.predict(X_test)

train_mae = np.mean(np.abs(X_train - train_preds_1), axis=(1, 2))
val_mae = np.mean(np.abs(X_val - val_preds_1), axis=(1, 2))
test_mae = np.mean(np.abs(X_test - test_preds_1), axis=(1, 2))

# Combine true labels and prediction scores for threshold optimization
all_labels = np.concatenate([y_train, y_val, y_test])
all_scores = np.concatenate([train_mae, val_mae, test_mae])

import seaborn as sns
# Add the predicted anomaly scores to the merged_df for correlation analysis
# We need to map the sequence-level MAE scores back to the original time points.

# Create a DataFrame for the sequence scores and their corresponding time indices
score_df = pd.DataFrame({
    'PatientID': [idx[0] for idx in time_idx],
    'DayRelativeToNearestHCT': [idx[1] for idx in time_idx],
    'AnomalyScore': all_scores
})

# Since multiple sequences can end on the same day, we might need to aggregate or take the score from the most recent sequence ending on that day. For simplicity here, we'll take the mean if multiple sequences end on the same day.
# A more sophisticated approach would be to align the score to the specific time point in the sequence.
# For now, let's merge based on PatientID and DayRelativeToNearestHCT, taking the first score if duplicates exist after merging, or perhaps the max/mean.
# Let's first ensure the score_df is sorted correctly to potentially align with the original merged_df time structure if needed, though a merge is generally safer.

# Sort the score_df just in case, though merge should handle matching keys.
score_df = score_df.sort_values(['PatientID', 'DayRelativeToNearestHCT'])

# Merge the anomaly scores back to the original merged_df structure
# We need to decide how to handle the case where a day might not be the end of a sequence, or a day is the end of multiple overlapping sequences.
# A simple approach is to merge and keep all rows from merged_df, adding the score if it exists for that (PatientID, Day). Days that weren't the end of a sequence won't have a direct score.
# Alternatively, we can only consider the days that were the *end* of a sequence for this correlation analysis. Let's go with the latter for a direct correlation.

# Filter merged_df to include only the rows corresponding to the end of sequences used in training/validation/testing
# We can do this by merging merged_df with score_df
correlation_df = pd.merge(merged_df, score_df, on=['PatientID', 'DayRelativeToNearestHCT'], how='inner')

# Now, calculate the correlation matrix
# We are interested in the correlation between 'AnomalyScore' and the clinical markers
clinical_markers = ['MaxTemperature', 'NeutrophilCount'] + [col for col in correlation_df.columns if 'Consistency' in col]

# Select the columns for correlation analysis
correlation_cols = ['AnomalyScore'] + clinical_markers

# Calculate the correlation matrix
correlation_matrix = correlation_df[correlation_cols].corr()

print("\nCorrelation Matrix between Predicted Anomaly Scores and Clinical Markers:")
print(correlation_matrix)

# Visualize the correlation matrix using a heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=.5)
plt.title('Correlation Matrix: Anomaly Score vs. Clinical Markers', fontsize=16)
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.savefig('correlation_matrix.pdf', dpi=600, format='pdf')
plt.show()

# You can also look specifically at the correlations with 'AnomalyScore'
print("\nCorrelation of Anomaly Score with Individual Clinical Markers:")
print(correlation_matrix['AnomalyScore'].drop('AnomalyScore'))

# You might also want to visualize scatter plots for the most interesting correlations
# Example: Scatter plot for AnomalyScore vs MaxTemperature
if 'MaxTemperature' in clinical_markers:
    plt.figure(figsize=(8, 6))
    sns.scatterplot(data=correlation_df, x='MaxTemperature', y='AnomalyScore', alpha=0.6)
    plt.title('Anomaly Score vs. Max Temperature', fontsize=16)
    plt.xlabel('Max Temperature (Scaled)', fontsize=12)
    plt.ylabel('Anomaly Score (MAE)', fontsize=12)
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.tight_layout()
    plt.savefig('scatter_anomaly_temperature.pdf', dpi=600, format='pdf')
    plt.show()

# Example: Scatter plot for AnomalyScore vs NeutrophilCount
if 'NeutrophilCount' in clinical_markers:
    plt.figure(figsize=(8, 6))
    sns.scatterplot(data=correlation_df, x='NeutrophilCount', y='AnomalyScore', alpha=0.6)
    plt.title('Anomaly Score vs. Neutrophil Count', fontsize=16)
    plt.xlabel('Neutrophil Count (Scaled)', fontsize=12)
    plt.ylabel('Anomaly Score (MAE)', fontsize=12)
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.tight_layout()
    plt.savefig('scatter_anomaly_neutrophil.pdf', dpi=600, format='pdf')
    plt.show()

# For one-hot encoded categorical features like Consistency, correlation is still calculated
# but interpretation differs from continuous variables. A heatmap is usually sufficient.

# Linear Classifier (Logistic Regression on reconstruction errors)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score

# Assuming you have reconstruction_errors_val and y_val

# Define the parameter grid for Logistic Regression
# Ensure compatible combinations of penalty and solver
param_grid = [
    {'C': [0.001, 0.01, 0.1, 1, 10, 100],
     'penalty': ['l1', 'l2'],
     'solver': ['liblinear', 'saga']}, # liblinear and saga support l1/l2
    {'C': [0.001, 0.01, 0.1, 1, 10, 100],
     'penalty': ['elasticnet'],
     'solver': ['saga'], # Only saga supports elasticnet
     'l1_ratio': [0.1, 0.5, 0.9]}, # l1_ratio is needed for elasticnet
     {'C': [0.001, 0.01, 0.1, 1, 10, 100],
      'penalty': [None], # Change 'none' to None (capitalized)
      'solver': ['saga', 'lbfgs']} # saga and lbfgs support None penalty
]

# Create a GridSearchCV object
grid_search = GridSearchCV(LogisticRegression(class_weight='balanced'), param_grid, cv=5, scoring='f1', n_jobs=-1, error_score='raise') # Using F1-score as the scoring metric

# Fit the grid search to the validation data
grid_search.fit(reconstruction_errors_val.reshape(-1, 1), y_val)

# Print the best hyperparameters and best score
print("Best hyperparameters:", grid_search.best_params_)
print("Best F1-score:", grid_search.best_score_)

# You can then use the best model found by grid search to evaluate on the test set
best_clf = grid_search.best_estimator_
predictions = best_clf.predict(reconstruction_errors_test.reshape(-1, 1))

# Evaluate on the test set as you did before
print(classification_report(y_test, predictions, zero_division=1))
print(roc_auc_score(y_test, best_clf.predict_proba(reconstruction_errors_test.reshape(-1, 1))[:, 1]))
print(average_precision_score(y_test, best_clf.predict_proba(reconstruction_errors_test.reshape(-1, 1))[:, 1]))

In [ ]:
# Predict using trained classifier on test set
predictions = best_clf.predict(reconstruction_errors_test.reshape(-1, 1))

# Individual Metrics
precision = precision_score(y_test, predictions,zero_division=1)
recall = recall_score(y_test, predictions)
f1 = f1_score(y_test, predictions)
roc_auc = roc_auc_score(y_test, reconstruction_errors_test)
pr_auc = average_precision_score(y_test, reconstruction_errors_test)
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}") # F1-score for the minority/positive class (Anomaly)
print(f"ROC AUC: {roc_auc:.4f}")
print(f"PR AUC: {pr_auc:.4f}")

# Calculate weighted average F1-score
weighted_f1 = f1_score(y_test, predictions, average='weighted',zero_division=1) #Overall summary
print(f"Weighted Average F1 Score: {weighted_f1:.4f}")

print("\nClassification Report on Test Set (Unsupervised Fashion):")
print(classification_report(y_test, predictions, zero_division=1))

In [ ]:
# prompt: Draw PR Curve from pr_auc
import os # Ensure os is imported
from sklearn.metrics import precision_recall_curve
import matplotlib.pyplot as plt

# Define the output directory for Logistic Regression plots
logistic_regression_output_dir = "/content/drive/MyDrive/Final_Implementations/DynaBiome_Figures/BenchMark_Figures/LogisticRegression"

# Create the directory if it doesn't exist
os.makedirs(logistic_regression_output_dir, exist_ok=True)


# Assuming 'reconstruction_errors' and 'true_labels' are defined as in your code
precision, recall, thresholds = precision_recall_curve(y_test, predictions)

plt.figure(figsize=(8, 6))
plt.plot(recall, precision, marker='.', label=f'PR Curve (AUC = {pr_auc:.2f})')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.legend()
plt.grid(True)
# Use the new directory variable
plt.savefig(f"{logistic_regression_output_dir}/PR_Curve_LR.pdf", dpi=600, bbox_inches='tight')
plt.show()

In [ ]:
from sklearn.metrics import roc_curve, precision_recall_curve
import matplotlib.pyplot as plt
logistic_regression_output_dir = "/content/drive/MyDrive/Final_Implementations/DynaBiome_Figures/LogisticRegression"
os.makedirs(logistic_regression_output_dir, exist_ok=True)

# Calculate fpr and tpr using roc_curve before plotting:
fpr, tpr, _ = roc_curve(y_test, predictions)  # Assuming 'y_test' and 'reconstruction_errors' are defined

# Now you can proceed with plotting the ROC curve:
plt.figure(figsize=(12, 5))

# --- ROC ---
plt.subplot(1, 2, 1)
plt.plot(fpr, tpr, label=f'ROC AUC = {roc_auc:.3f}')
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve")
plt.legend()
plt.grid(True)
# Save as high-resolution PDF using the defined directory variable
plt.savefig(f"{logistic_regression_output_dir}/ROC_Curve_LogisticRegression.pdf", dpi=600, bbox_inches='tight')

In [ ]:
from sklearn.metrics import roc_curve, precision_recall_curve, confusion_matrix
import seaborn as sns
# Calculate and assign the confusion matrix to 'cm'
cm = confusion_matrix(y_test, predictions) # Calculate confusion matrix

# Draw Confusion Matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=['Predicted Normal', 'Predicted Anomaly'],
            yticklabels=['Actual Normal', 'Actual Anomaly'])
plt.title('Confusion Matrix')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.savefig("/content/drive/MyDrive/Final_Implementations/DynaBiome_Figures/LogisticRegression/PR_Curve_LogisticRegression.pdf", dpi=600, bbox_inches='tight')
plt.show()

# Keras Tuner for MLP Hyperparameters

In [ ]:
# !pip install keras-tuner
# from kerastuner.tuners import RandomSearch

# # Function to tune hyperparameters
# def tune_mlp_model(hp):
#     model = Sequential([
#         Dense(hp.Int('neurons1', 32, 128, step=32), activation=hp.Choice('activation', ['relu', 'tanh']), input_shape=(1,)),
#         Dense(hp.Int('neurons2', 16, 64, step=16), activation=hp.Choice('activation', ['relu', 'tanh'])),
#         Dense(1, activation='sigmoid')
#     ])
#     model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
#     return model

# # Define tuner
# tuner = RandomSearch(
#     tune_mlp_model,
#     objective='val_accuracy',
#     max_trials=10,
#     executions_per_trial=1,
#     directory='mlp_tuner',
#     project_name='mlp_optimization'
# )

# # Search for best hyperparameters
# tuner.search(reconstruction_errors_val.reshape(-1, 1), y_val, epochs=50, validation_split=0.2)

# # Get the best hyperparameters
# best_hyperparameters = tuner.get_best_hyperparameters()[0]
# print("Best Hyperparameters:", best_hyperparameters.values)


# Non-Linear Classifier (MLP on reconstruction errors)

In [ ]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Input

# Use best hyperparameters found via tuning
input_layer = Input(shape=(1,))
dense_1 = Dense(96, activation='relu')(input_layer)  # Update neurons1 to 96
dense_2 = Dense(64, activation='relu')(dense_1)  # Update neurons2 to 64
output_layer = Dense(1, activation='sigmoid')(dense_2)

# Define the optimized model
mlp_model = Model(inputs=input_layer, outputs=output_layer)

# Compile the model
mlp_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model with the same dataset but use best hyperparameters
mlp_model.fit(reconstruction_errors_val.reshape(-1, 1), y_val, epochs=100, batch_size=16)  # Keep epochs & batch_size unchanged

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import classification_report, roc_curve, roc_auc_score, precision_recall_curve, average_precision_score, matthews_corrcoef # Import classification_report and other necessary functions

# Define the directory path where the plots will be saved
mlp_output_dir = "/content/drive/MyDrive/Final_Implementations/DynaBiome_Figures/MLP"

# Create the directory if it doesn't exist. exist_ok=True prevents an error if the directory already exists.
os.makedirs(mlp_output_dir, exist_ok=True)

# Get probabilities from the trained MLP model on the test set reconstruction errors
mlp_probs = mlp_model.predict(reconstruction_errors_test.reshape(-1, 1)).flatten()

# Convert probabilities to binary labels (you can adjust the threshold if needed)
threshold = 0.5
mlp_pred_classes = (mlp_probs > threshold).astype(int)

# Classification report
print("MLP Classification Report (Trained on Validation, Tested on Test):")
print(classification_report(y_test, mlp_pred_classes, target_names=["Non-Dysbiotic", "Dysbiotic"], zero_division=1))

# Compute ROC curve and AUC score
fpr_mlp, tpr_mlp, _ = roc_curve(y_test, mlp_probs)
roc_auc_mlp = roc_auc_score(y_test, mlp_probs)
print(f"MLP ROC AUC Score: {roc_auc_mlp}")

# Compute Precision-Recall curve
precision_mlp, recall_mlp, _ = precision_recall_curve(y_test, mlp_probs)
average_precision_mlp = average_precision_score(y_test, mlp_probs)
print(f"MLP Average Precision: {average_precision_mlp}")

# Plot ROC curve
plt.figure(figsize=(8, 5))
plt.plot(fpr_mlp, tpr_mlp, color='blue', lw=2, label=f'ROC curve (AUC = {roc_auc_mlp:.2f})')
plt.plot([0, 1], [0, 1], color='gray', linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve - MLP on Reconstruction Error (Trained Val, Tested Test)')
plt.legend(loc="lower right")
plt.savefig("/content/drive/MyDrive/Final_Implementations/DynaBiome_Figures/MLP/ROC_Curve_MLP.pdf", dpi=600, bbox_inches='tight')

plt.show()

# Plot Precision-Recall curve
plt.figure(figsize=(8, 5))
plt.plot(recall_mlp, precision_mlp, color='red', lw=2, label=f'PR curve (AP = {average_precision_mlp:.2f})')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve - MLP on Reconstruction Error (Trained Val, Tested Test)')
plt.legend(loc="lower left")
plt.savefig("/content/drive/MyDrive/Final_Implementations/DynaBiome_Figures/MLP/PR_Curve_MLP.pdf", dpi=600, bbox_inches='tight')
plt.show()

# Compute Matthews Correlation Coefficient (MCC)
mcc_mlp = matthews_corrcoef(y_test, mlp_pred_classes)
print(f"MLP Matthews Correlation Coefficient (MCC): {mcc_mlp}")

In [ ]:
# prompt: PRINT PR AUC of MLP

print(f"PR AUC of MLP: {average_precision_mlp:.4f}")


In [ ]:
# prompt: Compute Precision, recall and F1 Score on MLP

from sklearn.metrics import precision_score, recall_score, f1_score

# Assuming y_test and mlp_pred_classes are already defined from the preceding code

# Compute Precision
precision_mlp = precision_score(y_test, mlp_pred_classes, zero_division=1)

# Compute Recall
recall_mlp = recall_score(y_test, mlp_pred_classes, zero_division=1)

# Compute F1 Score
f1_mlp = f1_score(y_test, mlp_pred_classes, zero_division=1)

print(f"MLP Precision: {precision_mlp:.4f}")
print(f"MLP Recall: {recall_mlp:.4f}")
print(f"MLP F1 Score: {f1_mlp:.4f}")

In [ ]:
# prompt: Generate confusion matrix for mlp

# Compute confusion matrix for MLP predictions
cm_mlp = confusion_matrix(y_test, mlp_pred_classes)

# Display confusion matrix using seaborn heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(cm_mlp, annot=True, fmt="d", cmap="Blues",
            xticklabels=['Predicted Non-Dysbiotic', 'Predicted Dysbiotic'],
            yticklabels=['Actual Non-Dysbiotic', 'Actual Dysbiotic'])
plt.title('Confusion Matrix - MLP on Reconstruction Error (Trained Val, Tested Test)')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.savefig("/content/drive/MyDrive/Final_Implementations/DynaBiome_Figures/MLP/ConfusionMatrix_MLP.pdf", dpi=600, bbox_inches='tight')
plt.show()


In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import OneClassSVM
from sklearn.metrics import make_scorer, f1_score
import numpy as np # Import numpy

# Define custom scoring function
def custom_f1_scorer(estimator, X, y):
    anomaly_scores = estimator.decision_function(X)
    # In One-Class SVM, lower scores mean more anomalous.
    # We want to find a threshold such that a certain percentage of the *training* data
    # (which is assumed to be mostly normal) is considered anomalies.
    # The 5th percentile means the 5% lowest scores are considered anomalous.
    # Since decision_function gives higher values for inliers, we threshold on the low side.
    threshold = np.percentile(anomaly_scores, 5)
    anomaly_predictions = anomaly_scores < threshold # Anomalies are below the threshold
    # Only evaluate F1 on the actual anomalies in the test set.
    # For RandomizedSearchCV during validation, we score against y_val
    return f1_score(y, anomaly_predictions)


# Define the hyperparameter search space
param_grid = {
    'nu': np.linspace(0.01, 0.2, 10), # nu is an upper bound on the fraction of training errors
    'gamma': ['scale', 'auto', 0.01, 0.1, 1] # Kernel coefficient
}

# Set up RandomizedSearchCV
ocsvm = OneClassSVM(kernel="rbf")
# Use make_scorer with our custom F1 function, setting greater_is_better=True (default)
random_search = RandomizedSearchCV(ocsvm, param_grid, scoring=make_scorer(custom_f1_scorer), n_iter=10, cv=3, n_jobs=-1)

# *** Calculate reconstruction errors for the training data ***
# Assuming 'autoencoder' and 'X_train' are available from previous steps
X_train_pred = autoencoder.predict(X_train)
reconstruction_errors_train = np.mean(np.abs(X_train_pred - X_train), axis=(1, 2))


# Fit search on training data's reconstruction errors
# The y_train is used by the custom_f1_scorer during cross-validation
# to calculate the F1 score on the validation folds within the training data.
random_search.fit(reconstruction_errors_train.reshape(-1, 1), y_train)

# Show best parameters
print("Best Hyperparameters:", random_search.best_params_)

# Once Class SVM on reconstruction errors

In [ ]:
from sklearn.svm import OneClassSVM
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report, roc_curve, auc, precision_recall_curve
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler # Or StandardScaler

# 1. Train OCSVM on Reconstruction Errors of Normal Data
# Assuming 'reconstruction_errors_train' contains errors for normal training data

# 1. Calculate Reconstruction Errors for Training Data
X_train_pred = autoencoder.predict(X_train)  # Predict on training data
reconstruction_errors_train = np.mean(np.abs(X_train_pred - X_train), axis=(1, 2))

# Use best hyperparameters
best_nu = 0.0944
best_gamma = 0.01

# Train One-Class SVM on Reconstruction Errors of Normal Data
ocsvm = OneClassSVM(nu=best_nu, kernel="rbf", gamma=best_gamma)
ocsvm.fit(reconstruction_errors_train.reshape(-1, 1))

# 2. Predict Anomaly Scores on Test Data
anomaly_scores = ocsvm.decision_function(reconstruction_errors_test.reshape(-1, 1))
# Lower scores indicate higher anomaly probability

# 3. Thresholding for Anomaly Detection
threshold = np.percentile(anomaly_scores, 5)  # Adjust percentile for desired sensitivity
anomaly_predictions = anomaly_scores < threshold # Points below threshold are anomalies

# 4. Evaluate Performance and Print Classification Report
print("\nOne-Class SVM Classification Report:")
print(classification_report(y_test, anomaly_predictions))

# Define the output directory for SVM plots
svm_output_dir = "/content/drive/MyDrive/Final_Implementations/DynaBiome_Figures/svm"

# Create the directory if it doesn't exist
os.makedirs(svm_output_dir, exist_ok=True)

# 5. Plot ROC Curve
fpr, tpr, _ = roc_curve(y_test, -anomaly_scores) # Use negative scores for ROC (higher score = more anomalous)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic for One-Class SVM')
plt.legend(loc="lower right")
plt.savefig("/content/drive/MyDrive/Final_Implementations/DynaBiome_Figures/svm/RoC_Curve_SVM.pdf", dpi=600, bbox_inches='tight')
plt.show()

# 6. Plot PR Curve
precision, recall, _ = precision_recall_curve(y_test, -anomaly_scores) # Use negative scores for PR (higher score = more anomalous)
pr_auc = auc(recall, precision)

plt.figure(figsize=(8, 6))
plt.plot(recall, precision, color='darkorange', lw=2, label=f'PR curve (area = {pr_auc:.2f})')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve for One-Class SVM')
plt.legend(loc="lower left")
plt.savefig("/content/drive/MyDrive/Final_Implementations/DynaBiome_Figures/svm/PR_Curve_SVM.pdf", dpi=600, bbox_inches='tight')
plt.show()

In [ ]:
# prompt: Create confusion matrix for one class SVM Classification
from sklearn.metrics import confusion_matrix # Import confusion_matrix here
import seaborn as sns # Import seaborn for heatmap visualization
import matplotlib.pyplot as plt # Import matplotlib for plotting

# Calculate and assign the confusion matrix for One-Class SVM to 'cm_ocsvm'
cm_ocsvm = confusion_matrix(y_test, anomaly_predictions)

# Draw Confusion Matrix for One-Class SVM
plt.figure(figsize=(8, 6))
sns.heatmap(cm_ocsvm, annot=True, fmt="d", cmap="Blues",
            xticklabels=['Predicted Normal', 'Predicted Anomaly'],
            yticklabels=['Actual Normal', 'Actual Anomaly'])
plt.title('Confusion Matrix - One-Class SVM (Scaled Errors)')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.savefig("/content/drive/MyDrive/Final_Implementations/DynaBiome_Figures/svm/confusion_matrix.pdf", dpi=600, bbox_inches='tight')
plt.show()


In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.neighbors import KNeighborsClassifier

# Define KNN classifier
knn = KNeighborsClassifier()

# Corrected hyperparameter search space
param_grid = {
    'n_neighbors': [3, 5, 7, 9, 11, 15],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan', 'minkowski'],
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute']
}

# Perform Randomized Search
random_search = RandomizedSearchCV(knn, param_grid, cv=3, scoring='accuracy', n_iter=10, n_jobs=-1)
random_search.fit(reconstruction_errors_val.reshape(-1, 1), y_val)

# Print best parameters
print("Best Hyperparameters:", random_search.best_params_)


# K-Nearest Neighbors Classifier

In [ ]:
# prompt: Add K-Nearest Neighbors Classifier

from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, roc_auc_score, average_precision_score
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, precision_recall_curve
from sklearn.metrics import confusion_matrix # Import confusion_matrix
import seaborn as sns # Import seaborn

# Assuming you have already computed:
# reconstruction_errors_val, y_val
# reconstruction_errors_test, y_test

# Train K-Nearest Neighbors Classifier
# You might want to tune the 'n_neighbors' parameter
#knn = KNeighborsClassifier(n_neighbors=5)
# Use the best hyperparameters found
knn = KNeighborsClassifier(weights='distance', n_neighbors=11, metric='minkowski', algorithm='kd_tree')

# **Corrected:** Train on validation set reconstruction errors and labels
knn.fit(reconstruction_errors_val.reshape(-1, 1), y_val)

# Predict probabilities on Test Set Reconstruction Errors
# Use the predict_proba method on the trained KNN model
knn_probs = knn.predict_proba(reconstruction_errors_test.reshape(-1, 1))[:, 1]

# Get binary predictions using a threshold (default is 0.5 for predict_proba)
knn_preds = (knn_probs > 0.5).astype(int)

# Evaluation on Test Set
print("\nK-Nearest Neighbors Classification Report (Trained on Validation, Tested on Test):\n",
      classification_report(y_test, knn_preds, digits=4, zero_division=1))

# Compute ROC AUC
# Use the probabilities for AUC calculation
roc_auc_knn = roc_auc_score(y_test, knn_probs)
print("K-Nearest Neighbors ROC AUC (Trained on Validation, Tested on Test):", roc_auc_knn)

# Compute Average Precision (PR AUC)
# Use the probabilities for PR AUC calculation
average_precision_knn = average_precision_score(y_test, knn_probs)
print("K-Nearest Neighbors Average Precision (PR AUC) (Trained on Validation, Tested on Test):", average_precision_knn)

# Calculate Confusion Matrix for KNN
cm_knn = confusion_matrix(y_test, knn_preds)

# Draw Confusion Matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm_knn, annot=True, fmt="d", cmap="Blues",
            xticklabels=['Predicted Non-Dysbiotic', 'Predicted Dysbiotic'],
            yticklabels=['Actual Non-Dysbiotic', 'Actual Dysbiotic'])
plt.title('Confusion Matrix - K-Nearest Neighbors on Reconstruction Error (Trained Val, Tested Test)')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
#plt.savefig("/content/drive/MyDrive/Final_Implementations/DynaBiome_Figures/K_Nearest_Neighbor/confusion_matrix.pdf", dpi=600, bbox_inches='tight')
plt.show()


# Plot ROC curve
fpr_knn, tpr_knn, _ = roc_curve(y_test, knn_probs)

plt.figure(figsize=(8, 5))
plt.plot(fpr_knn, tpr_knn, color='green', lw=2, label=f'KNN ROC curve (AUC = {roc_auc_knn:.2f})')
plt.plot([0, 1], [0, 1], color='gray', linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve - K-Nearest Neighbors on Reconstruction Error (Trained Val, Tested Test)')
plt.legend(loc="lower right")
plt.grid(True)
#plt.savefig("/content/drive/MyDrive/Final_Implementations/DynaBiome_Figures/K_Nearest_Neighbor/KNN_ROC_curve.pdf", dpi=600, bbox_inches='tight')
plt.show()

# Plot Precision-Recall curve
precision_knn, recall_knn, _ = precision_recall_curve(y_test, knn_probs)

plt.figure(figsize=(8, 5))
plt.plot(recall_knn, precision_knn, color='purple', lw=2, label=f'KNN PR curve (AP = {average_precision_knn:.2f})')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve - K-Nearest Neighbors on Reconstruction Error (Trained Val, Tested Test)')
plt.legend(loc="lower left")
plt.grid(True)
#plt.savefig("/content/drive/MyDrive/Final_Implementations/DynaBiome_Figures/K_Nearest_Neighbor/PR_curve.pdf", dpi=600, bbox_inches='tight')
plt.show()

In [ ]:
!pip install xgboost

In [ ]:
from xgboost import cv, DMatrix

# Prepare data in DMatrix format
dtrain = DMatrix(reconstruction_errors_val.reshape(-1, 1), label=y_val)

# Set parameter space
xgb_params = {
    'max_depth': 5,
    'learning_rate': 0.1,
    'objective': 'binary:logistic',
    'eval_metric': 'logloss',
}

# Perform Cross-Validation
cv_results = cv(xgb_params, dtrain, num_boost_round=200, nfold=3, metrics="logloss", early_stopping_rounds=10)

# Find best number of trees
best_n_estimators = cv_results.shape[0]
print(f"Best number of estimators: {best_n_estimators}")


In [ ]:
# %%
#!pip install xgboost
# %%
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, roc_auc_score, roc_curve, precision_recall_curve, average_precision_score
import matplotlib.pyplot as plt
import seaborn as sns  # Import seaborn

# Train XGBoost on Validation Set Reconstruction Errors
xgb = XGBClassifier(eval_metric='logloss', n_estimators=200)

# **Corrected:** Train on validation set reconstruction errors
xgb.fit(reconstruction_errors_val.reshape(-1, 1), y_val)

# Predict probabilities on Test Set Reconstruction Errors
xgb_probs = xgb.predict_proba(reconstruction_errors_test.reshape(-1, 1))[:, 1]
xgb_preds = (xgb_probs > 0.5).astype(int)

# Evaluation on Test Set
print("XGBoost Classification Report (Trained on Validation, Tested on Test):\n",
      classification_report(y_test, xgb_preds, digits=4, zero_division=1))
print("XGBoost ROC AUC (Trained on Validation, Tested on Test):",
      roc_auc_score(y_test, xgb_probs))
print("XGBoost Average Precision (PR AUC) (Trained on Validation, Tested on Test):",
      average_precision_score(y_test, xgb_probs))

# Calculate Confusion Matrix for XGBoost
cm_xgb = confusion_matrix(y_test, xgb_preds)

# Draw Confusion Matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm_xgb, annot=True, fmt="d", cmap="Blues",
            xticklabels=['Predicted Non-Dysbiotic', 'Predicted Dysbiotic'],
            yticklabels=['Actual Non-Dysbiotic', 'Actual Dysbiotic'])
plt.title('Confusion Matrix - XGBoost on Reconstruction Error (Trained Val, Tested Test)')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
#plt.savefig("/Final_Implementations/DynaBiome_Figures/xgboost/confusion_matrix.pdf", dpi=600, bbox_inches='tight')
plt.show()

# Plot ROC curve
fpr_xgb, tpr_xgb, _ = roc_curve(y_test, xgb_probs)
roc_auc_xgb = roc_auc_score(y_test, xgb_probs) # Recalculate AUC using test probabilities

plt.figure(figsize=(8, 5))
plt.plot(fpr_xgb, tpr_xgb, color='darkorange', lw=2, label=f'XGBoost ROC curve (AUC = {roc_auc_xgb:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic for XGBoost (Trained Val, Tested Test)')
plt.legend(loc="lower right")
plt.grid(True)
#plt.savefig("/Final_Implementations/DynaBiome_Figures/xgboost/RoC_Curve.pdf", dpi=600, bbox_inches='tight')
plt.show()

# Plot Precision-Recall curve
precision_xgb, recall_xgb, _ = precision_recall_curve(y_test, xgb_probs)
pr_auc_xgb = average_precision_score(y_test, xgb_probs) # Recalculate PR AUC using test probabilities

plt.figure(figsize=(8, 6))
plt.plot(recall_xgb, precision_xgb, color='darkorange', lw=2, label=f'XGBoost PR curve (AP = {pr_auc_xgb:.2f})')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve for XGBoost (Trained Val, Tested Test)')
plt.legend(loc="lower left")
plt.grid(True)
#plt.savefig("/Final_Implementations/DynaBiome_Figures/xgboost/PR_Curve.pdf", dpi=600, bbox_inches='tight')
plt.show()

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier

# Define Random Forest classifier
rf = RandomForestClassifier(random_state=42)

# Corrected hyperparameter search space (removed 'weights')
param_grid = {
    'n_estimators': [50, 100, 200, 300],  # Number of trees
    'max_depth': [None, 10, 20, 30],  # Maximum depth
    'min_samples_split': [2, 5, 10],  # Minimum samples required to split
    'min_samples_leaf': [1, 2, 5],  # Minimum samples in a leaf node
    'max_features': ['sqrt', 'log2'],  # Number of features per tree
    'bootstrap': [True, False]  # Whether to bootstrap samples
}

# Perform Randomized Search
random_search = RandomizedSearchCV(rf, param_grid, cv=3, scoring='accuracy', n_iter=10, n_jobs=-1)
random_search.fit(reconstruction_errors_val.reshape(-1, 1), y_val)

# Print best parameters
print("Best Hyperparameters:", random_search.best_params_)


# Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score, average_precision_score
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, precision_recall_curve
from sklearn.metrics import confusion_matrix # Import confusion_matrix
import seaborn as sns # Import seaborn

# Assuming you have already computed reconstruction_errors_val, y_val,
# reconstruction_errors_test, and y_test from previous steps.

# Train Random Forest on Validation Set Reconstruction Errors
# You might want to tune the 'n_estimators' and other parameters
# rf = RandomForestClassifier(n_estimators=100, random_state=42)
# Use best hyperparameters found
rf = RandomForestClassifier(
    n_estimators=200,
    min_samples_split=5,
    min_samples_leaf=5,
    max_features='log2',
    max_depth=10,
    bootstrap=False,
    random_state=42
)

rf.fit(reconstruction_errors_val.reshape(-1, 1), y_val) # Train on validation errors and labels

# Predict probabilities on Test Set Reconstruction Errors
rf_probs = rf.predict_proba(reconstruction_errors_test.reshape(-1, 1))[:, 1]
rf_preds = (rf_probs > 0.5).astype(int) # Use a threshold (e.g., 0.5) to get binary predictions

# Evaluation on Test Set
print("Random Forest Classification Report (Trained on Validation, Tested on Test):\n",
      classification_report(y_test, rf_preds, digits=4, zero_division=1))
print("Random Forest ROC AUC (Trained on Validation, Tested on Test):",
      roc_auc_score(y_test, rf_probs))
print("Random Forest Average Precision (PR AUC) (Trained on Validation, Tested on Test):",
      average_precision_score(y_test, rf_probs))

# Calculate Confusion Matrix for Random Forest
cm_rf = confusion_matrix(y_test, rf_preds)

# Draw Confusion Matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm_rf, annot=True, fmt="d", cmap="Blues",
            xticklabels=['Predicted Non-Dysbiotic', 'Predicted Dysbiotic'],
            yticklabels=['Actual Non-Dysbiotic', 'Actual Dysbiotic'])
plt.title('Confusion Matrix - Random Forest on Reconstruction Error (Trained Val, Tested Test)')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
#plt.savefig("/content/drive/MyDrive/Final_Implementations/DynaBiome_Figures/RF/confusion_matrix.pdf", dpi=600, bbox_inches='tight')
plt.show()


# Plot ROC curve
fpr_rf, tpr_rf, _ = roc_curve(y_test, rf_probs)
roc_auc_rf = roc_auc_score(y_test, rf_probs) # Recalculate ROC AUC using test probabilities

plt.figure(figsize=(8, 5))
plt.plot(fpr_rf, tpr_rf, color='darkorange', lw=2, label=f'Random Forest ROC curve (AUC = {roc_auc_rf:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic for Random Forest (Trained Val, Tested Test)')
plt.legend(loc="lower right")
plt.grid(True)
#plt.savefig("/content/drive/MyDrive/Final_Implementations/DynaBiome_Figures/RF/RoC_Curve.pdf", dpi=600, bbox_inches='tight')
plt.show()

# Plot Precision-Recall curve
precision_rf, recall_rf, _ = precision_recall_curve(y_test, rf_probs)
average_precision_rf = average_precision_score(y_test, rf_probs) # Recalculate PR AUC using test probabilities

plt.figure(figsize=(8, 6))
plt.plot(recall_rf, precision_rf, color='darkorange', lw=2, label=f'Random Forest PR curve (AP = {average_precision_rf:.2f})')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve for Random Forest (Trained Val, Tested Test)')
plt.legend(loc="lower left")
plt.grid(True)
#plt.savefig("/content/drive/MyDrive/Final_Implementations/DynaBiome_Figures/RF/PR_Curve.pdf", dpi=600, bbox_inches='tight')
plt.show()

<a name="ensemble-learning"></a>
# Ensemble Learning on the Classifiers

## Averaged Probabilities (Soft Voting)

In [ ]:
# Assuming the following trained models are available from your notebook:
# best_clf (Logistic Regression - trained on reconstruction_errors_val)
# mlp_model (Keras MLP - trained on reconstruction_errors_val)
# ocsvm (One-Class SVM - trained on reconstruction_errors_train, decision_function on test)
# knn (K-Nearest Neighbors - trained on reconstruction_errors_val)
# xgb (XGBoost - trained on reconstruction_errors_val)
# rf (Random Forest - trained on reconstruction_errors_val)

# Ensure reconstruction_errors_test is available and reshaped if needed for models expecting (n_samples, 1) input
reconstruction_errors_test_reshaped = reconstruction_errors_test.reshape(-1, 1)

# 1. Logistic Regression Probabilities
lr_probs = best_clf.predict_proba(reconstruction_errors_test_reshaped)[:, 1]

# 2. Keras MLP Probabilities
# Make sure the MLP model is loaded if necessary
# from tensorflow.keras.models import load_model
# mlp_model = load_model("your_mlp_model.h5") # Load if not already in memory
mlp_probs = mlp_model.predict(reconstruction_errors_test_reshaped).flatten()

# 3. KNN Probabilities
knn_probs = knn.predict_proba(reconstruction_errors_test_reshaped)[:, 1]

# 4. XGBoost Probabilities
xgb_probs = xgb.predict_proba(reconstruction_errors_test_reshaped)[:, 1]

# 5. Random Forest Probabilities
rf_probs = rf.predict_proba(reconstruction_errors_test_reshaped)[:, 1]

# 6. One-Class SVM Scores
# OCSVM's decision_function gives a score. Lower scores are more anomalous.
# For soft voting, we need a probability-like score where higher indicates anomaly.
# We can use the negative of the decision function.
ocsvm_scores = -ocsvm.decision_function(reconstruction_errors_test_reshaped).flatten()

# You might consider scaling the OCSVM scores to be on a similar scale as probabilities (0-1)
# For instance, using MinMaxScaler if needed, but it's not strictly necessary for simple averaging.
# from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler()
# ocsvm_scaled_scores = scaler.fit_transform(ocsvm_scores.reshape(-1, 1)).flatten()
# We'll use the raw negative scores for now in the ensemble calculation.

In [ ]:
import numpy as np

# Combine probabilities (excluding OCSVM for simple averaging with probability-based models)
# If you want to include OCSVM, you would need a way to convert its score to a probability or
# use a different ensembling approach that handles scores and probabilities.
# Let's average the probabilities from LR, MLP, KNN, XGBoost, and RF for a start.
averaged_probs = np.mean([lr_probs, mlp_probs, knn_probs, xgb_probs, rf_probs], axis=0)

# You can set a threshold on the averaged probabilities to get binary predictions
ensemble_predictions = (averaged_probs > 0.5).astype(int)

In [ ]:
from sklearn.metrics import classification_report, roc_auc_score, average_precision_score, confusion_matrix
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, precision_recall_curve
import seaborn as sns

# Classification Report
print("Ensemble (Averaged Probabilities) Classification Report on Test Set:")
print(classification_report(y_test, ensemble_predictions, target_names=["Non-Dysbiotic", "Dysbiotic"], zero_division=1))

# Compute ROC AUC
roc_auc_ensemble = roc_auc_score(y_test, averaged_probs)
print(f"Ensemble ROC AUC Score: {roc_auc_ensemble:.4f}")

# Compute Average Precision (PR AUC)
average_precision_ensemble = average_precision_score(y_test, averaged_probs)
print(f"Ensemble Average Precision (PR AUC): {average_precision_ensemble:.4f}")

# Calculate Confusion Matrix
cm_ensemble = confusion_matrix(y_test, ensemble_predictions)

# Draw Confusion Matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm_ensemble, annot=True, fmt="d", cmap="Blues",
            xticklabels=['Predicted Non-Dysbiotic', 'Predicted Dysbiotic'],
            yticklabels=['Actual Non-Dysbiotic', 'Actual Dysbiotic'])
plt.title('Confusion Matrix - Ensemble (Averaged Probabilities)')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
#plt.savefig("/content/drive/MyDrive/Final_Implementations/DynaBiome_Figures/Ensemble_Avg/confusion_matrix.pdf", dpi=600, bbox_inches='tight')
plt.show()

# Plot ROC curve
fpr_ensemble, tpr_ensemble, _ = roc_curve(y_test, averaged_probs)

plt.figure(figsize=(8, 5))
plt.plot(fpr_ensemble, tpr_ensemble, color='blue', lw=2, label=f'Ensemble ROC curve (AUC = {roc_auc_ensemble:.2f})')
plt.plot([0, 1], [0, 1], color='gray', linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve - Ensemble (Averaged Probabilities)')
plt.legend(loc="lower right")
plt.grid(True)
#plt.savefig("/content/drive/MyDrive/Final_Implementations/DynaBiome_Figures/Ensemble_Avg/RoC_Curve.pdf", dpi=600, bbox_inches='tight')
plt.show()

# Plot Precision-Recall curve
precision_ensemble, recall_ensemble, _ = precision_recall_curve(y_test, averaged_probs)

plt.figure(figsize=(8, 5))
plt.plot(recall_ensemble, precision_ensemble, color='red', lw=2, label=f'Ensemble PR curve (AP = {average_precision_ensemble:.2f})')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve - Ensemble (Averaged Probabilities)')
plt.legend(loc="lower left")
plt.grid(True)
#plt.savefig("/content/drive/MyDrive/Final_Implementations/DynaBiome_Figures/Ensemble_Avg/PR_Curve.pdf", dpi=600, bbox_inches='tight')
plt.show()

## Weighted Mechanism (Weighted Averaging) (Soft Voting)



In [ ]:
from sklearn.ensemble import VotingClassifier
# Assuming you have the scikit-learn versions of your trained models:
# best_clf (Logistic Regression)
# sklearn_mlp (if you train a scikit-learn MLP)
# knn (K-Nearest Neighbors)
# xgb (XGBoost)
# rf (Random Forest)

# For One-Class SVM, you would typically not include it in a standard soft voting classifier
# because its decision_function is not directly comparable to probability outputs of others.
# If you want to include its influence, you might use hard voting or a custom ensembling approach.

estimators_sklearn = [
    ('lr', best_clf),
    ('knn', knn),
    ('xgb', xgb),
    ('rf', rf)
    # Add sklearn_mlp if available
    # ('mlp', sklearn_mlp)
]

# Create the Voting Classifier with soft voting
voting_clf = VotingClassifier(estimators=estimators_sklearn, voting='soft', weights=[1, 1, 1, 1]) # Adjust weights as needed

# Train the Voting Classifier (This trains the individual models if not already trained, but
# since yours are trained, it essentially just sets up the combination)
# However, typically, you train the individual models first and then use the VotingClassifier
# to combine their predictions without retraining.
# To use the already trained models' predictions directly in VotingClassifier for evaluation:
# You would fit the VotingClassifier on a small dummy dataset just to enable prediction,
# or more correctly, collect the predictions and combine them manually as shown in the previous steps.

# Let's stick to the manual averaging of probabilities as it directly uses your existing trained models' outputs.
# If you were to use VotingClassifier's fit method, it would re-train the estimators,
# which is not what you want if you've already tuned and trained them on the validation data.

# The manual averaging method shown above is the correct way to ensemble
# the predictions of your already trained models on the test set.

In [ ]:
# Example of weighted averaging (adjust weights based on your analysis)
# You would need to determine these weights based on the performance of each model on the validation set (e.g., F1-score, ROC AUC).
# For example, if XGBoost had the highest F1 on validation, give it a higher weight.
weights = [1.2, 0.8, 1.5, 1.0, 1.1] # Example weights for LR, MLP, KNN, XGBoost, RF
weighted_averaged_probs = np.average([lr_probs, mlp_probs, knn_probs, xgb_probs, rf_probs], axis=0, weights=weights)

weighted_ensemble_predictions = (weighted_averaged_probs > 0.5).astype(int)

# Evaluate the weighted ensemble similarly to the unweighted one.
print("\nWeighted Ensemble Classification Report on Test Set:")
print(classification_report(y_test, weighted_ensemble_predictions, target_names=["Non-Dysbiotic", "Dysbiotic"], zero_division=1))

In [ ]:
# prompt: calculate and plot metrics for the weighted ensemble

# Compute ROC AUC for weighted ensemble
roc_auc_weighted_ensemble = roc_auc_score(y_test, weighted_averaged_probs)
print(f"Weighted Ensemble ROC AUC Score: {roc_auc_weighted_ensemble:.4f}")

# Compute Average Precision (PR AUC) for weighted ensemble
average_precision_weighted_ensemble = average_precision_score(y_test, weighted_averaged_probs)
print(f"Weighted Ensemble Average Precision (PR AUC): {average_precision_weighted_ensemble:.4f}")

# Calculate Confusion Matrix for weighted ensemble
cm_weighted_ensemble = confusion_matrix(y_test, weighted_ensemble_predictions)

# Define output directory for Weighted Ensemble plots
weighted_ensemble_output_dir = "/content/drive/MyDrive/Final_Implementations/DynaBiome_Figures/Ensemble_Weighted"
os.makedirs(weighted_ensemble_output_dir, exist_ok=True)


# Draw Confusion Matrix for weighted ensemble
plt.figure(figsize=(8, 6))
sns.heatmap(cm_weighted_ensemble, annot=True, fmt="d", cmap="Blues",
            xticklabels=['Predicted Non-Dysbiotic', 'Predicted Dysbiotic'],
            yticklabels=['Actual Non-Dysbiotic', 'Actual Dysbiotic'])
plt.title('Confusion Matrix - Weighted Ensemble')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.savefig(f"{weighted_ensemble_output_dir}/confusion_matrix.pdf", dpi=600, bbox_inches='tight')
plt.show()

# Plot ROC curve for weighted ensemble
fpr_weighted_ensemble, tpr_weighted_ensemble, _ = roc_curve(y_test, weighted_averaged_probs)

plt.figure(figsize=(8, 5))
plt.plot(fpr_weighted_ensemble, tpr_weighted_ensemble, color='green', lw=2, label=f'Weighted Ensemble ROC curve (AUC = {roc_auc_weighted_ensemble:.2f})')
plt.plot([0, 1], [0, 1], color='gray', linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve - Weighted Ensemble')
plt.legend(loc="lower right")
plt.grid(True)
plt.savefig(f"{weighted_ensemble_output_dir}/RoC_Curve.pdf", dpi=600, bbox_inches='tight')
plt.show()

# Plot Precision-Recall curve for weighted ensemble
precision_weighted_ensemble, recall_weighted_ensemble, _ = precision_recall_curve(y_test, weighted_averaged_probs)

plt.figure(figsize=(8, 5))
plt.plot(recall_weighted_ensemble, precision_weighted_ensemble, color='purple', lw=2, label=f'Weighted Ensemble PR curve (AP = {average_precision_weighted_ensemble:.2f})')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve - Weighted Ensemble')
plt.legend(loc="lower left")
plt.grid(True)
plt.savefig(f"{weighted_ensemble_output_dir}/PR_Curve.pdf", dpi=600, bbox_inches='tight')
plt.show()


In [ ]:
# prompt: Print RoC (AUC) & PR (AUC) of weighted esemble

print(f"RoC (AUC) of Weighted Ensemble: {roc_auc_weighted_ensemble:.4f}")
print(f"PR (AUC) of Weighted Ensemble: {average_precision_weighted_ensemble:.4f}")


# Advanced Ensembling: Stacking

## Stacking with LogisticRegression

In [ ]:
# Generate Out-of-Fold Predictions on the Validation Set
from sklearn.model_selection import StratifiedKFold
import numpy as np

# Assuming you have reconstruction_errors_val and y_val

# Number of folds for cross-validation
n_splits = 5
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Initialize arrays to store out-of-fold predictions
# Each column will be the predictions from one base model
oof_preds = np.zeros((reconstruction_errors_val.shape[0], 5)) # 5 columns for LR, MLP, KNN, XGB, RF

# List of base models (excluding OCSVM for now as its output is a score, not probability)
base_models = [
    ('lr', best_clf),
    ('mlp', mlp_model), # We'll handle Keras prediction separately
    ('knn', knn),
    ('xgb', xgb),
    ('rf', rf)
]

# Iterate through folds and generate out-of-fold predictions
for fold, (train_index, val_index) in enumerate(skf.split(reconstruction_errors_val.reshape(-1, 1), y_val)):
    print(f"Processing Fold {fold + 1}/{n_splits}")

    X_train_fold, X_val_fold = reconstruction_errors_val[train_index].reshape(-1, 1), reconstruction_errors_val[val_index].reshape(-1, 1)
    y_train_fold, y_val_fold = y_val[train_index], y_val[val_index]

    for i, (name, model) in enumerate(base_models):
        if name == 'mlp':
            # For Keras model, train on the fold data
            # Need to clone and retrain the model for each fold or save/load weights
            # A simpler approach for demonstration: assuming the trained mlp_model is sufficient
            # and we generate predictions on the val_index for this fold.
            # In a proper implementation, you would train a new Keras model on X_train_fold
            # and predict on X_val_fold.
            fold_mlp_preds = model.predict(X_val_fold).flatten()
            oof_preds[val_index, i] = fold_mlp_preds
        else:
            # Train a clone of the scikit-learn model on the fold data
            from sklearn.base import clone
            cloned_model = clone(model)
            cloned_model.fit(X_train_fold, y_train_fold)
            oof_preds[val_index, i] = cloned_model.predict_proba(X_val_fold)[:, 1]

# oof_preds now contains the out-of-fold predictions for each base model on the validation set

In [ ]:
# Combine test predictions from base models
test_preds = np.vstack([lr_probs, mlp_probs, knn_probs, xgb_probs, rf_probs]).T

# test_preds now has shape (n_test_samples, n_base_models)

In [ ]:
from sklearn.linear_model import LogisticRegression

# Define the meta-model
meta_model = LogisticRegression()

# Train the meta-model on the out-of-fold predictions
meta_model.fit(oof_preds, y_val)

In [ ]:
# Make final predictions on the test data using the meta-model
stacked_predictions_probs = meta_model.predict_proba(test_preds)[:, 1]

# Convert probabilities to binary predictions
stacked_predictions = (stacked_predictions_probs > 0.5).astype(int)

In [ ]:
from sklearn.metrics import classification_report, roc_auc_score, average_precision_score, confusion_matrix
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, precision_recall_curve
import seaborn as sns

# Classification Report
print("Stacked Ensemble Classification Report on Test Set:")
print(classification_report(y_test, stacked_predictions, target_names=["Non-Dysbiotic", "Dysbiotic"], zero_division=1))

# Compute ROC AUC
roc_auc_stacked = roc_auc_score(y_test, stacked_predictions_probs)
print(f"Stacked Ensemble ROC AUC Score: {roc_auc_stacked:.4f}")

# Compute Average Precision (PR AUC)
average_precision_stacked = average_precision_score(y_test, stacked_predictions_probs)
print(f"Stacked Ensemble Average Precision (PR AUC): {average_precision_stacked:.4f}")

# Calculate Confusion Matrix
cm_stacked = confusion_matrix(y_test, stacked_predictions)

# Draw Confusion Matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm_stacked, annot=True, fmt="d", cmap="Blues",
            xticklabels=['Predicted Non-Dysbiotic', 'Predicted Dysbiotic'],
            yticklabels=['Actual Non-Dysbiotic', 'Actual Anomaly'])
plt.title('Confusion Matrix - Stacked Ensemble')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()

# Plot ROC curve
fpr_stacked, tpr_stacked, _ = roc_curve(y_test, stacked_predictions_probs)

plt.figure(figsize=(8, 5))
plt.plot(fpr_stacked, tpr_stacked, color='blue', lw=2, label=f'Stacked ROC curve (AUC = {roc_auc_stacked:.2f})')
plt.plot([0, 1], [0, 1], color='gray', linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve - Stacked Ensemble')
plt.legend(loc="lower right")
plt.grid(True)
#plt.savefig("/content/drive/MyDrive/Final_Implementations/DynaBiome_Figures/Ensemble_Stacking/RoC_Curve.pdf", dpi=600, bbox_inches='tight')
plt.show()

# Plot Precision-Recall curve
precision_stacked, recall_stacked, _ = precision_recall_curve(y_test, stacked_predictions_probs)

plt.figure(figsize=(8, 5))
plt.plot(recall_stacked, precision_stacked, color='red', lw=2, label=f'Stacked PR curve (AP = {average_precision_stacked:.2f})')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve - Stacked Ensemble')
plt.legend(loc="lower left")
plt.grid(True)
#plt.savefig("/content/drive/MyDrive/Final_Implementations/DynaBiome_Figures/Ensemble_Stacking/PR_Curve.pdf", dpi=600, bbox_inches='tight')
plt.show()

## Stacking with XGBoost Meta-Model

In [ ]:
from xgboost import XGBClassifier

# Define the XGBoost meta-model
# You might want to tune the hyperparameters of the meta-model as well
meta_model_xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)

# Train the meta-model on the out-of-fold predictions
meta_model_xgb.fit(oof_preds, y_val)

In [ ]:
# Make final predictions on the test data using the XGBoost meta-model
stacked_predictions_probs_xgb = meta_model_xgb.predict_proba(test_preds)[:, 1]

# Convert probabilities to binary predictions
stacked_predictions_xgb = (stacked_predictions_probs_xgb > 0.5).astype(int)

In [ ]:
from sklearn.metrics import classification_report, roc_auc_score, average_precision_score, confusion_matrix
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, precision_recall_curve
import seaborn as sns

# Classification Report
print("Stacked Ensemble (XGBoost Meta-Model) Classification Report on Test Set:")
print(classification_report(y_test, stacked_predictions_xgb, target_names=["Non-Dysbiotic", "Dysbiotic"], zero_division=1))

# Compute ROC AUC
roc_auc_stacked_xgb = roc_auc_score(y_test, stacked_predictions_probs_xgb)
print(f"Stacked Ensemble (XGBoost Meta-Model) ROC AUC Score: {roc_auc_stacked_xgb:.4f}")

# Compute Average Precision (PR AUC)
average_precision_stacked_xgb = average_precision_score(y_test, stacked_predictions_probs_xgb)
print(f"Stacked Ensemble (XGBoost Meta-Model) Average Precision (PR AUC): {average_precision_stacked_xgb:.4f}")

# Calculate Confusion Matrix
cm_stacked_xgb = confusion_matrix(y_test, stacked_predictions_xgb)

# Draw Confusion Matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm_stacked_xgb, annot=True, fmt="d", cmap="Blues",
            xticklabels=['Predicted Non-Dysbiotic', 'Predicted Dysbiotic'],
            yticklabels=['Actual Non-Dysbiotic', 'Actual Anomaly'])
plt.title('Confusion Matrix - Stacked Ensemble (XGBoost Meta-Model)')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
#plt.savefig("/content/drive/MyDrive/Final_Implementations/DynaBiome_Figures/Ensemble_Stacking_XGB/confusion_matrix.pdf", dpi=600, bbox_inches='tight')
plt.show()

# Plot ROC curve
fpr_stacked_xgb, tpr_stacked_xgb, _ = roc_curve(y_test, stacked_predictions_probs_xgb)

plt.figure(figsize=(8, 5))
plt.plot(fpr_stacked_xgb, tpr_stacked_xgb, color='blue', lw=2, label=f'Stacked (XGBoost) ROC curve (AUC = {roc_auc_stacked_xgb:.2f})')
plt.plot([0, 1], [0, 1], color='gray', linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve - Stacked Ensemble (XGBoost Meta-Model)')
plt.legend(loc="lower right")
plt.grid(True)
#plt.savefig("/content/drive/MyDrive/Final_Implementations/DynaBiome_Figures/Ensemble_Stacking_XGB/RoC_Curve.pdf", dpi=600, bbox_inches='tight')
plt.show()

# Plot Precision-Recall curve
precision_stacked_xgb, recall_stacked_xgb, _ = precision_recall_curve(y_test, stacked_predictions_probs_xgb)

plt.figure(figsize=(8, 5))
plt.plot(recall_stacked_xgb, precision_stacked_xgb, color='red', lw=2, label=f'Stacked (XGBoost) PR curve (AP = {average_precision_stacked_xgb:.2f})')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve - Stacked Ensemble (XGBoost Meta-Model)')
plt.legend(loc="lower left")
plt.grid(True)
#plt.savefig("/content/drive/MyDrive/Final_Implementations/DynaBiome_Figures/Ensemble_Stacking_XGB/PR_Curve.pdf", dpi=600, bbox_inches='tight')
plt.show()

In [ ]:
# prompt: prompt: There is too much overlap between the lines in the ROC (AUC) and PR(AUC). Fix it

import matplotlib.pyplot as plt
# --- B. Combined ROC and PR Plotting ---

# Store ROC and PR curve data for each model
roc_curves_data = {}
pr_curves_data = {}

# Function to collect ROC and PR data
def collect_curve_data(model_name, y_true, y_probs_or_scores, color, linestyle='-'):
    """Collects ROC and PR curve data for a given model's scores/probabilities."""
    # For ROC curve, higher score/prob means higher true positive rate at lower false positive rate
    # For PR curve, higher score/prob means higher precision at higher recall
    # sklearn's roc_curve and precision_recall_curve work correctly with scores directly.

    fpr, tpr, _ = roc_curve(y_true, y_probs_or_scores)
    roc_auc = roc_auc_score(y_true, y_probs_or_scores)
    roc_curves_data[model_name] = {'fpr': fpr, 'tpr': tpr, 'auc': roc_auc, 'color': color, 'linestyle': linestyle}

    precision, recall, _ = precision_recall_curve(y_true, y_probs_or_scores)
    pr_auc = average_precision_score(y_true, y_probs_or_scores)
    pr_curves_data[model_name] = {'precision': precision, 'recall': recall, 'auc': pr_auc, 'color': color, 'linestyle': linestyle}

# Collect data for Original Model (LSTM-AE with optimal threshold)
# Use reconstruction errors as the score directly. Higher error -> more likely anomaly.
collect_curve_data(
    'LSTM-AE (Reconstruction Error)',
    y_test,
    reconstruction_errors_test,
    'darkorange',
    '-'
)

# Collect data for Individual Classifiers (assuming probabilities are available)
# Use predict_proba[:, 1] for standard classifiers
# For OCSVM, use negative scores as the "probability" or score for AUC
collect_curve_data('Logistic Regression', y_test, best_clf.predict_proba(reconstruction_errors_test.reshape(-1, 1))[:, 1], 'blue', '--')
collect_curve_data('MLP', y_test, mlp_model.predict(reconstruction_errors_test.reshape(-1, 1)).flatten(), 'green', '--')
# For OCSVM, use negative decision_function scores. Lower scores are anomalous, so negative makes higher values anomalous.
collect_curve_data('One-Class SVM', y_test, -ocsvm.decision_function(reconstruction_errors_test.reshape(-1, 1)).flatten(), 'red', '--')
collect_curve_data('K-Nearest Neighbors', y_test, knn.predict_proba(reconstruction_errors_test.reshape(-1, 1))[:, 1], 'purple', '--')
collect_curve_data('XGBoost', y_test, xgb.predict_proba(reconstruction_errors_test.reshape(-1, 1))[:, 1], 'brown', '--')
collect_curve_data('Random Forest', y_test, rf.predict_proba(reconstruction_errors_test.reshape(-1, 1))[:, 1], 'pink', '--')


# Collect data for Ensemble Methods (using the averaged/stacked probabilities)
collect_curve_data('Averaged Probabilities Ensemble', y_test, averaged_probs, 'cyan', '-')
collect_curve_data('Weighted Ensemble', y_test, weighted_averaged_probs, 'magenta', '-')
# Use the stacked ensemble probabilities as scores
collect_curve_data('Stacked Ensemble (LR Meta)', y_test, meta_model.predict_proba(test_preds)[:, 1], 'gray', '-')
collect_curve_data('Stacked Ensemble (XGB Meta)', y_test, meta_model_xgb.predict_proba(test_preds)[:, 1], 'olive', '-')


# --- Plotting ---

output_dir_evaluation_plots = "/content/drive/MyDrive/Final_Implementations/DynaBiome_Figures/Combined_Evaluation_Plots/"
os.makedirs(output_dir_evaluation_plots, exist_ok=True)

# Figure 1: ROC Curves
plt.figure(figsize=(10, 8))
for model_name, data in roc_curves_data.items():
    plt.plot(data['fpr'], data['tpr'], color=data['color'], linestyle=data['linestyle'],
             label=f'{model_name} (AUC = {data["auc"]:.3f})')

plt.plot([0, 1], [0, 1], 'k--', lw=1)  # Diagonal line
plt.xlabel("False Positive Rate", fontsize=12)
plt.ylabel("True Positive Rate", fontsize=12)
plt.title("Receiver Operating Characteristic (ROC) Curves", fontsize=14)
plt.legend(loc="lower right", fontsize=10)
plt.grid(True, linestyle='--', alpha=0.6)
plt.tight_layout()
plt.savefig(f"{output_dir_evaluation_plots}/Combined_ROC_Curves.pdf", dpi=600, bbox_inches='tight')
plt.show()

# Figure 2: PR Curves
plt.figure(figsize=(10, 8))
for model_name, data in pr_curves_data.items():
    plt.plot(data['recall'], data['precision'], color=data['color'], linestyle=data['linestyle'],
             label=f'{model_name} (AP = {data["auc"]:.3f})')

plt.xlabel("Recall", fontsize=12)
plt.ylabel("Precision", fontsize=12)
plt.title("Precision-Recall (PR) Curves", fontsize=14)
plt.legend(loc="lower left", fontsize=10)
plt.grid(True, linestyle='--', alpha=0.6)
plt.ylim([0.0, 1.05]) # Standard practice for PR curves
plt.xlim([0.0, 1.0])
plt.tight_layout()
plt.savefig(f"{output_dir_evaluation_plots}/Combined_PR_Curves.pdf", dpi=600, bbox_inches='tight')
plt.show()


# RoC (AUC) as original vs All Classifiers except ensemble learning

In [ ]:
import matplotlib.pyplot as plt
# --- Plotting ROC Curve for Original Model vs. Individual Classifiers (Excluding Ensemble) ---

# Define the output directory
output_dir_comparison_plots = "/content/drive/MyDrive/Final_Implementations/DynaBiome_Figures/Comparison_Plots/"
os.makedirs(output_dir_comparison_plots, exist_ok=True)


plt.figure(figsize=(10, 8))

# Plot ROC Curve for the Original Model (LSTM-AE Reconstruction Error)
# Use reconstruction errors as the score directly. Higher error -> more likely anomaly.
fpr_original, tpr_original, _ = roc_curve(y_test, reconstruction_errors_test)
roc_auc_original = roc_auc_score(y_test, reconstruction_errors_test)
plt.plot(fpr_original, tpr_original, color='darkorange', lw=2, label=f'LSTM-AE (Reconstruction Error) (AUC = {roc_auc_original:.3f})')


# Plot ROC Curves for Individual Classifiers (trained on reconstruction errors)
# Use predict_proba[:, 1] for standard classifiers. For OCSVM, use negative scores.

# Logistic Regression
fpr_lr, tpr_lr, _ = roc_curve(y_test, best_clf.predict_proba(reconstruction_errors_test.reshape(-1, 1))[:, 1])
roc_auc_lr = roc_auc_score(y_test, best_clf.predict_proba(reconstruction_errors_test.reshape(-1, 1))[:, 1])
plt.plot(fpr_lr, tpr_lr, color='blue', linestyle='--', lw=2, label=f'Logistic Regression (AUC = {roc_auc_lr:.3f})')

# MLP
mlp_probs_test = mlp_model.predict(reconstruction_errors_test.reshape(-1, 1)).flatten()
fpr_mlp, tpr_mlp, _ = roc_curve(y_test, mlp_probs_test)
roc_auc_mlp = roc_auc_score(y_test, mlp_probs_test)
plt.plot(fpr_mlp, tpr_mlp, color='green', linestyle='--', lw=2, label=f'MLP (AUC = {roc_auc_mlp:.3f})')

# One-Class SVM
ocsvm_scores_test = -ocsvm.decision_function(reconstruction_errors_test.reshape(-1, 1)).flatten()
fpr_ocsvm, tpr_ocsvm, _ = roc_curve(y_test, ocsvm_scores_test)
roc_auc_ocsvm = roc_auc_score(y_test, ocsvm_scores_test)
plt.plot(fpr_ocsvm, tpr_ocsvm, color='red', linestyle='--', lw=2, label=f'One-Class SVM (AUC = {roc_auc_ocsvm:.3f})')

# K-Nearest Neighbors
knn_probs_test = knn.predict_proba(reconstruction_errors_test.reshape(-1, 1))[:, 1]
fpr_knn, tpr_knn, _ = roc_curve(y_test, knn_probs_test)
roc_auc_knn = roc_auc_score(y_test, knn_probs_test)
plt.plot(fpr_knn, tpr_knn, color='purple', linestyle='--', lw=2, label=f'K-Nearest Neighbors (AUC = {roc_auc_knn:.3f})')

# XGBoost
xgb_probs_test = xgb.predict_proba(reconstruction_errors_test.reshape(-1, 1))[:, 1]
fpr_xgb, tpr_xgb, _ = roc_curve(y_test, xgb_probs_test)
roc_auc_xgb = roc_auc_score(y_test, xgb_probs_test)
plt.plot(fpr_xgb, tpr_xgb, color='brown', linestyle='--', lw=2, label=f'XGBoost (AUC = {roc_auc_xgb:.3f})')

# Random Forest
rf_probs_test = rf.predict_proba(reconstruction_errors_test.reshape(-1, 1))[:, 1]
fpr_rf, tpr_rf, _ = roc_curve(y_test, rf_probs_test)
roc_auc_rf = roc_auc_score(y_test, rf_probs_test)
plt.plot(fpr_rf, tpr_rf, color='pink', linestyle='--', lw=2, label=f'Random Forest (AUC = {roc_auc_rf:.3f})')


# Plot the diagonal reference line
plt.plot([0, 1], [0, 1], 'k--', lw=1)

plt.xlabel("False Positive Rate", fontsize=12)
plt.ylabel("True Positive Rate", fontsize=12)
plt.title("ROC Curves: LSTM-AE vs. Individual Classifiers", fontsize=14)
plt.legend(loc="lower right", fontsize=10)
plt.grid(True, linestyle='--', alpha=0.6)
plt.tight_layout()
plt.savefig(f"{output_dir_comparison_plots}/ROC_Curve_Original_vs_Individual_Classifiers.pdf", dpi=600, bbox_inches='tight')
plt.show()


# RoC (AUC) as original vs all ensemble learning only

In [ ]:
# prompt: Generate RoC (AUC) as original vs all ensemble learning only

import matplotlib.pyplot as plt
# --- Plotting ROC Curve for Original Model vs. Ensemble Learning ---

# Define the output directory for comparison plots if not already defined
output_dir_comparison_plots = "/content/drive/MyDrive/Final_Implementations/DynaBiome_Figures/Comparison_Plots/"
os.makedirs(output_dir_comparison_plots, exist_ok=True)

plt.figure(figsize=(10, 8))

# Plot ROC Curve for the Original Model (LSTM-AE Reconstruction Error)
# Use reconstruction errors as the score directly. Higher error -> more likely anomaly.
fpr_original, tpr_original, _ = roc_curve(y_test, reconstruction_errors_test)
roc_auc_original = roc_auc_score(y_test, reconstruction_errors_test)
plt.plot(fpr_original, tpr_original, color='darkorange', lw=2, label=f'LSTM-AE (Reconstruction Error) (AUC = {roc_auc_original:.3f})')


# Plot ROC Curves for Ensemble Models

# Averaged Probabilities Ensemble
fpr_avg_ensemble, tpr_avg_ensemble, _ = roc_curve(y_test, averaged_probs)
roc_auc_avg_ensemble = roc_auc_score(y_test, averaged_probs)
plt.plot(fpr_avg_ensemble, tpr_avg_ensemble, color='cyan', linestyle='-', lw=2, label=f'Averaged Probabilities Ensemble (AUC = {roc_auc_avg_ensemble:.3f})')

# Weighted Ensemble
fpr_weighted_ensemble, tpr_weighted_ensemble, _ = roc_curve(y_test, weighted_averaged_probs)
roc_auc_weighted_ensemble = roc_auc_score(y_test, weighted_averaged_probs)
plt.plot(fpr_weighted_ensemble, tpr_weighted_ensemble, color='magenta', linestyle='-', lw=2, label=f'Weighted Ensemble (AUC = {roc_auc_weighted_ensemble:.3f})')

# Stacked Ensemble (Logistic Regression Meta-Model)
# Assuming test_preds contains the stacked inputs for test data
stacked_predictions_probs_lr_meta = meta_model.predict_proba(test_preds)[:, 1]
fpr_stacked_lr, tpr_stacked_lr, _ = roc_curve(y_test, stacked_predictions_probs_lr_meta)
roc_auc_stacked_lr = roc_auc_score(y_test, stacked_predictions_probs_lr_meta)
plt.plot(fpr_stacked_lr, tpr_stacked_lr, color='gray', linestyle='-', lw=2, label=f'Stacked Ensemble (LR Meta) (AUC = {roc_auc_stacked_lr:.3f})')

# Stacked Ensemble (XGBoost Meta-Model)
# Assuming test_preds contains the stacked inputs for test data
stacked_predictions_probs_xgb_meta = meta_model_xgb.predict_proba(test_preds)[:, 1]
fpr_stacked_xgb, tpr_stacked_xgb, _ = roc_curve(y_test, stacked_predictions_probs_xgb_meta)
roc_auc_stacked_xgb = roc_auc_score(y_test, stacked_predictions_probs_xgb_meta)
plt.plot(fpr_stacked_xgb, tpr_stacked_xgb, color='olive', linestyle='-', lw=2, label=f'Stacked Ensemble (XGB Meta) (AUC = {roc_auc_stacked_xgb:.3f})')


# Plot the diagonal reference line
plt.plot([0, 1], [0, 1], 'k--', lw=1)

plt.xlabel("False Positive Rate", fontsize=12)
plt.ylabel("True Positive Rate", fontsize=12)
plt.title("ROC Curves: LSTM-AE vs. Ensemble Classifiers", fontsize=14)
plt.legend(loc="lower right", fontsize=10)
plt.grid(True, linestyle='--', alpha=0.6)
plt.tight_layout()
plt.savefig(f"{output_dir_comparison_plots}/ROC_Curve_Original_vs_Ensemble_Classifiers.pdf", dpi=600, bbox_inches='tight')
plt.show()


In [ ]:
# prompt: Generate PR (AUC) of original vs All Classifiers except ensemble learning

import matplotlib.pyplot as plt
# --- Plotting PR Curve for Original Model vs. Individual Classifiers (Excluding Ensemble) ---

# Define the output directory if not already defined
output_dir_comparison_plots = "/content/drive/MyDrive/Final_Implementations/DynaBiome_Figures/Comparison_Plots/"
os.makedirs(output_dir_comparison_plots, exist_ok=True)

plt.figure(figsize=(10, 8))

# Plot PR Curve for the Original Model (LSTM-AE Reconstruction Error)
# Use reconstruction errors as the score directly. Higher error -> more likely anomaly.
precision_original, recall_original, _ = precision_recall_curve(y_test, reconstruction_errors_test)
pr_auc_original = average_precision_score(y_test, reconstruction_errors_test)
plt.plot(recall_original, precision_original, color='darkorange', lw=2, label=f'LSTM-AE (Reconstruction Error) (AP = {pr_auc_original:.3f})')

# Plot PR Curves for Individual Classifiers (trained on reconstruction errors)
# Use predict_proba[:, 1] for standard classifiers. For OCSVM, use negative scores.

# Logistic Regression
precision_lr, recall_lr, _ = precision_recall_curve(y_test, best_clf.predict_proba(reconstruction_errors_test.reshape(-1, 1))[:, 1])
pr_auc_lr = average_precision_score(y_test, best_clf.predict_proba(reconstruction_errors_test.reshape(-1, 1))[:, 1])
plt.plot(recall_lr, precision_lr, color='blue', linestyle='--', lw=2, label=f'Logistic Regression (AP = {pr_auc_lr:.3f})')

# MLP
mlp_probs_test = mlp_model.predict(reconstruction_errors_test.reshape(-1, 1)).flatten()
precision_mlp, recall_mlp, _ = precision_recall_curve(y_test, mlp_probs_test)
pr_auc_mlp = average_precision_score(y_test, mlp_probs_test)
plt.plot(recall_mlp, precision_mlp, color='green', linestyle='--', lw=2, label=f'MLP (AP = {pr_auc_mlp:.3f})')

# One-Class SVM
ocsvm_scores_test = -ocsvm.decision_function(reconstruction_errors_test.reshape(-1, 1)).flatten()
precision_ocsvm, recall_ocsvm, _ = precision_recall_curve(y_test, ocsvm_scores_test)
pr_auc_ocsvm = average_precision_score(y_test, ocsvm_scores_test)
plt.plot(recall_ocsvm, precision_ocsvm, color='red', linestyle='--', lw=2, label=f'One-Class SVM (AP = {pr_auc_ocsvm:.3f})')

# K-Nearest Neighbors
knn_probs_test = knn.predict_proba(reconstruction_errors_test.reshape(-1, 1))[:, 1]
precision_knn, recall_knn, _ = precision_recall_curve(y_test, knn_probs_test)
pr_auc_knn = average_precision_score(y_test, knn_probs_test)
plt.plot(recall_knn, precision_knn, color='purple', linestyle='--', lw=2, label=f'K-Nearest Neighbors (AP = {pr_auc_knn:.3f})')

# XGBoost
xgb_probs_test = xgb.predict_proba(reconstruction_errors_test.reshape(-1, 1))[:, 1]
precision_xgb, recall_xgb, _ = precision_recall_curve(y_test, xgb_probs_test)
pr_auc_xgb = average_precision_score(y_test, xgb_probs_test)
plt.plot(recall_xgb, precision_xgb, color='brown', linestyle='--', lw=2, label=f'XGBoost (AP = {pr_auc_xgb:.3f})')

# Random Forest
rf_probs_test = rf.predict_proba(reconstruction_errors_test.reshape(-1, 1))[:, 1]
precision_rf, recall_rf, _ = precision_recall_curve(y_test, rf_probs_test)
pr_auc_rf = average_precision_score(y_test, rf_probs_test)
plt.plot(recall_rf, precision_rf, color='pink', linestyle='--', lw=2, label=f'Random Forest (AP = {pr_auc_rf:.3f})')


plt.xlabel("Recall", fontsize=12)
plt.ylabel("Precision", fontsize=12)
plt.title("Precision-Recall Curves: LSTM-AE vs. Individual Classifiers", fontsize=14)
plt.legend(loc="lower left", fontsize=10)
plt.grid(True, linestyle='--', alpha=0.6)
plt.ylim([0.0, 1.05])
plt.xlim([0.0, 1.0])
plt.tight_layout()
plt.savefig(f"{output_dir_comparison_plots}/PR_Curve_Original_vs_Individual_Classifiers.pdf", dpi=600, bbox_inches='tight')
plt.show()


<a name="statistical-analysis"></a>
## Statistical Test

## Statistical Evaluation and Bootstrap Analysis

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.utils import resample
from scipy.stats import ttest_rel, wilcoxon
# from statsmodels.sandbox.stats.runs import mcnemar # Keeping this import, although statsmodels.stats.contingency_tables.mcnemar is used
from statsmodels.stats.contingency_tables import mcnemar
from typing import Dict, Any, List
import seaborn as sns # Ensure seaborn is imported for plotting
from sklearn.metrics import roc_auc_score, average_precision_score, f1_score, roc_curve, precision_recall_curve # Ensure all metrics are imported
import os

# Assuming y_test, reconstruction_errors_test, anomaly_predictions, best_clf, mlp_model, ocsvm, knn, xgb, rf are available from previous cells

# Store results in dictionaries
metric_distributions: Dict[str, Dict[str, List[float]]] = {}
metric_summary: Dict[str, Dict[str, Dict[str, Any]]] = {}
statistical_test_results: Dict[str, Dict[str, Dict[str, float]]] = {}

# Define the models and their corresponding probabilities and binary predictions
models = {
    'LSTM-AE': {'probs': reconstruction_errors_test, 'preds': anomaly_predictions}, # Assuming reconstruction_errors_test is the "score" for LSTM-AE
    'Logistic Regression': {'probs': best_clf.predict_proba(reconstruction_errors_test.reshape(-1, 1))[:, 1], 'preds': best_clf.predict(reconstruction_errors_test.reshape(-1, 1))},
    'MLP': {'probs': mlp_model.predict(reconstruction_errors_test.reshape(-1, 1)).flatten(), 'preds': (mlp_model.predict(reconstruction_errors_test.reshape(-1, 1)).flatten() > 0.5).astype(int)},
    'One-Class SVM': {'probs': -ocsvm.decision_function(reconstruction_errors_test.reshape(-1, 1)).flatten(), 'preds': (ocsvm.decision_function(reconstruction_errors_test.reshape(-1, 1)).flatten() < 0).astype(int)}, # Using negative scores for AUC/PR AUC, and thresholding at 0 for binary predictions
    'KNN': {'probs': knn.predict_proba(reconstruction_errors_test.reshape(-1, 1))[:, 1], 'preds': knn.predict(reconstruction_errors_test.reshape(-1, 1))},
    'XGBoost': {'probs': xgb.predict_proba(reconstruction_errors_test.reshape(-1, 1))[:, 1], 'preds': xgb.predict(reconstruction_errors_test.reshape(-1, 1))},
    'Random Forest': {'probs': rf.predict_proba(reconstruction_errors_test.reshape(-1, 1))[:, 1], 'preds': rf.predict(reconstruction_errors_test.reshape(-1, 1))},
    # Add ensemble models if desired
    'Averaged Ensemble': {'probs': averaged_probs, 'preds': ensemble_predictions},
    'Weighted Ensemble': {'probs': weighted_averaged_probs, 'preds': weighted_ensemble_predictions},
    'Stacked (LR Meta)': {'probs': stacked_predictions_probs, 'preds': stacked_predictions},
    'Stacked (XGB Meta)': {'probs': stacked_predictions_probs_xgb, 'preds': stacked_predictions_xgb},
}

n_iterations = 1000
alpha = 0.05 # For confidence intervals and statistical significance

print("Performing Bootstrap Testing...")
for model_name, data in models.items():
    model_probs = data['probs']
    model_preds = data['preds']

    roc_aucs = []
    pr_aucs = []
    f1_scores = []

    # Handle potential issues with OCSVM or other models having constant predictions/scores
    # that would prevent AUC/F1 calculation on some bootstrapped samples.
    # For simplicity, we'll skip iterations where metrics fail.
    successful_iterations = 0
    # Use a fixed random state for reproducibility of bootstrap samples themselves
    bootstrap_rng = np.random.default_rng(seed=42)

    # Store (y_true_sample, probs_sample) for plotting later
    bootstrap_samples_for_plotting: List[tuple[np.ndarray, np.ndarray]] = []


    while successful_iterations < n_iterations:
        # Resample with replacement
        indices = bootstrap_rng.choice(np.arange(len(y_test)), size=len(y_test), replace=True)

        y_true_sample = y_test[indices]
        probs_sample = model_probs[indices]
        preds_sample = model_preds[indices]

        try:
            # Compute metrics for the sample
            # Check for multiple classes in true labels and variation in scores/predictions
            if len(np.unique(y_true_sample)) > 1:
                 if len(np.unique(probs_sample)) > 1:
                    roc_auc_sample = roc_auc_score(y_true_sample, probs_sample)
                    pr_auc_sample = average_precision_score(y_true_sample, probs_sample)
                    roc_aucs.append(roc_auc_sample)
                    pr_aucs.append(pr_auc_sample)
                 # We still try to compute F1 even if probs are constant, as preds might vary
                 if len(np.unique(preds_sample)) > 1:
                      f1_score_sample = f1_score(y_true_sample, preds_sample, zero_division=0)
                      f1_scores.append(f1_score_sample)
            # If only one class in y_true_sample, metrics like AUC, PR AUC, and standard F1 are not well-defined.
            # We append np.nan or skip, skipping is better for distribution calculations later.

            # Store sample for plotting if metrics were attempted (even if they failed)
            # This ensures the lists for plotting match the iterations where we *tried* to compute metrics
            bootstrap_samples_for_plotting.append((y_true_sample, probs_sample))

            successful_iterations += 1 # Increment only if sample was generated and attempted

        except Exception as e:
            # print(f"Skipping iteration {successful_iterations} for {model_name} due to error: {e}")
            # Still store the sample even if metric calculation failed, for consistent plotting sample count
            bootstrap_samples_for_plotting.append((y_true_sample, probs_sample))
            successful_iterations += 1 # Increment even on error to match total iterations


    # Store distributions
    metric_distributions[model_name] = {
        'ROC AUC': roc_aucs,
        'PR AUC': pr_aucs,
        'F1 Score': f1_scores,
        'bootstrap_samples_for_plotting': bootstrap_samples_for_plotting # Store samples for plotting
    }

    # Compute mean and 95% CI
    metric_summary[model_name] = {}
    for metric_name, values in {
        'ROC AUC': roc_aucs,
        'PR AUC': pr_aucs,
        'F1 Score': f1_scores
        }.items():
        if values: # Ensure list is not empty
            mean_val = np.mean(values)
            # Compute confidence interval using percentile method
            lower_bound = np.percentile(values, (alpha / 2) * 100)
            upper_bound = np.percentile(values, 100 - (alpha / 2) * 100)
            metric_summary[model_name][metric_name] = {
                'mean': mean_val,
                'ci_lower': lower_bound,
                'ci_upper': upper_bound,
                'distribution': values # Keep distribution for later plotting
            }
        else:
             metric_summary[model_name][metric_name] = {
                'mean': np.nan,
                'ci_lower': np.nan,
                'ci_upper': np.nan,
                'distribution': []
            }


print("\nPerforming Statistical Testing...")
# Collect bootstrap distributions for LSTM-AE explicitly after the loop finishes
lstm_ae_roc_auc_dist = metric_distributions['LSTM-AE']['ROC AUC']
lstm_ae_pr_auc_dist = metric_distributions['LSTM-AE']['PR AUC']
lstm_ae_f1_dist = metric_distributions['LSTM-AE']['F1 Score']
lstm_ae_preds = models['LSTM-AE']['preds'] # Binary predictions from LSTM-AE thresholding

statistical_test_results['vs LSTM-AE'] = {}

# Statistical Testing Against LSTM-AE

In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import ttest_rel, wilcoxon
from statsmodels.stats.contingency_tables import mcnemar

for model_name, data in models.items():
    if model_name == 'LSTM-AE':
        continue  # Skip comparison with itself

    model_roc_auc_dist = metric_distributions[model_name]['ROC AUC']
    model_pr_auc_dist = metric_distributions[model_name]['PR AUC']
    model_f1_dist = metric_distributions[model_name]['F1 Score']
    model_preds = data['preds']

    statistical_test_results['vs LSTM-AE'][model_name] = {}

    # Ensure lengths match for paired tests
    min_len_roc = min(len(lstm_ae_roc_auc_dist), len(model_roc_auc_dist))
    if min_len_roc >= 2:
        t_stat_roc, p_value_t_roc = ttest_rel(lstm_ae_roc_auc_dist[:min_len_roc], model_roc_auc_dist[:min_len_roc])
        statistical_test_results['vs LSTM-AE'][model_name]['ROC AUC (T-test p)'] = p_value_t_roc

        # Wilcoxon test fix: Ensure at least 2 distinct values
        unique_values_lstm = len(set(lstm_ae_roc_auc_dist[:min_len_roc]))
        unique_values_model = len(set(model_roc_auc_dist[:min_len_roc]))
        if min_len_roc >= 2 and unique_values_lstm > 1 and unique_values_model > 1:
            try:
                w_stat_roc, p_value_w_roc = wilcoxon(lstm_ae_roc_auc_dist[:min_len_roc], model_roc_auc_dist[:min_len_roc])
                statistical_test_results['vs LSTM-AE'][model_name]['ROC AUC (Wilcoxon p)'] = p_value_w_roc
            except ValueError:
                statistical_test_results['vs LSTM-AE'][model_name]['ROC AUC (Wilcoxon p)'] = np.nan
        else:
            statistical_test_results['vs LSTM-AE'][model_name]['ROC AUC (Wilcoxon p)'] = np.nan
    else:
        statistical_test_results['vs LSTM-AE'][model_name]['ROC AUC (T-test p)'] = np.nan
        statistical_test_results['vs LSTM-AE'][model_name]['ROC AUC (Wilcoxon p)'] = np.nan

    # Repeat fixes for PR AUC and F1 Score
    for metric_name, lstm_dist, model_dist in zip(
        ['PR AUC', 'F1 Score'],
        [lstm_ae_pr_auc_dist, lstm_ae_f1_dist],
        [model_pr_auc_dist, model_f1_dist]
    ):
        min_len = min(len(lstm_dist), len(model_dist))
        if min_len >= 2:
            t_stat, p_value_t = ttest_rel(lstm_dist[:min_len], model_dist[:min_len])
            statistical_test_results['vs LSTM-AE'][model_name][f'{metric_name} (T-test p)'] = p_value_t

            # Wilcoxon test fix
            unique_values_lstm = len(set(lstm_dist[:min_len]))
            unique_values_model = len(set(model_dist[:min_len]))
            if min_len >= 2 and unique_values_lstm > 1 and unique_values_model > 1:
                try:
                    w_stat, p_value_w = wilcoxon(lstm_dist[:min_len], model_dist[:min_len])
                    statistical_test_results['vs LSTM-AE'][model_name][f'{metric_name} (Wilcoxon p)'] = p_value_w
                except ValueError:
                    statistical_test_results['vs LSTM-AE']

# Statistical Comparison of Models

In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import ttest_rel, wilcoxon, mannwhitneyu
from statsmodels.stats.contingency_tables import mcnemar

# Define the ensemble models
ensemble_models = {
    'Averaged Ensemble': {'probs': averaged_probs, 'preds': ensemble_predictions},
    'Weighted Ensemble': {'probs': weighted_averaged_probs, 'preds': weighted_ensemble_predictions},
    'Stacked (LR Meta)': {'probs': stacked_predictions_probs, 'preds': stacked_predictions},
    'Stacked (XGB Meta)': {'probs': stacked_predictions_probs_xgb, 'preds': stacked_predictions_xgb},
}

# Store statistical test results for ensemble comparisons
ensemble_statistical_test_results: Dict[str, Dict[str, Dict[str, float]]] = {}

print("\nPerforming Statistical Testing Among Ensemble Learners...")

ensemble_model_names = list(ensemble_models.keys())

# Iterate through all unique pairs of ensemble models
for i in range(len(ensemble_model_names)):
    for j in range(i + 1, len(ensemble_model_names)):
        model1_name = ensemble_model_names[i]
        model2_name = ensemble_model_names[j]

        model1_data = ensemble_models[model1_name]
        model2_data = ensemble_models[model2_name]

        model1_roc_auc_dist = np.array(metric_distributions[model1_name]['ROC AUC'], dtype=np.float64)
        model1_pr_auc_dist = np.array(metric_distributions[model1_name]['PR AUC'], dtype=np.float64)
        model1_f1_dist = np.array(metric_distributions[model1_name]['F1 Score'], dtype=np.float64)
        model1_preds = model1_data['preds']

        model2_roc_auc_dist = np.array(metric_distributions[model2_name]['ROC AUC'], dtype=np.float64)
        model2_pr_auc_dist = np.array(metric_distributions[model2_name]['PR AUC'], dtype=np.float64)
        model2_f1_dist = np.array(metric_distributions[model2_name]['F1 Score'], dtype=np.float64)
        model2_preds = model2_data['preds']

        comparison_key = f'{model1_name} vs {model2_name}'
        ensemble_statistical_test_results[comparison_key] = {}

        # Perform paired tests on metric distributions (ROC AUC, PR AUC, F1 Score)
        for metric_name, dist1, dist2 in zip(
            ['ROC AUC', 'PR AUC', 'F1 Score'],
            [model1_roc_auc_dist, model1_pr_auc_dist, model1_f1_dist],
            [model2_roc_auc_dist, model2_pr_auc_dist, model2_f1_dist]
        ):
            min_len = min(len(dist1), len(dist2))
            if min_len >= 2:
                # Paired t-test
                t_stat, p_value_t = ttest_rel(dist1[:min_len], dist2[:min_len])
                ensemble_statistical_test_results[comparison_key][f'{metric_name} (T-test p)'] = p_value_t

                # Wilcoxon test
                if len(set(dist1[:min_len])) > 1 and len(set(dist2[:min_len])) > 1:
                    try:
                        w_stat, p_value_w = wilcoxon(dist1[:min_len], dist2[:min_len])
                        ensemble_statistical_test_results[comparison_key][f'{metric_name} (Wilcoxon p)'] = p_value_w
                    except ValueError:
                        ensemble_statistical_test_results[comparison_key][f'{metric_name} (Wilcoxon p)'] = np.nan
                else:
                    ensemble_statistical_test_results[comparison_key][f'{metric_name} (Wilcoxon p)'] = np.nan

                 # Mann-Whitney U test (alternative)
                u_stat, p_value_u = mannwhitneyu(dist1[:min_len], dist2[:min_len])
                ensemble_statistical_test_results[comparison_key][f'{metric_name} (Mann-Whitney p)'] = p_value_u


            else:
                ensemble_statistical_test_results[comparison_key][f'{metric_name} (T-test p)'] = np.nan
                ensemble_statistical_test_results[comparison_key][f'{metric_name} (Wilcoxon p)'] = np.nan
                ensemble_statistical_test_results[comparison_key][f'{metric_name} (Mann-Whitney p)'] = np.nan


        # McNemar test on binary predictions
        contingency_table_mcnemar = pd.crosstab(model1_preds, model2_preds).reindex(
            pd.MultiIndex.from_product([[0, 1], [0, 1]], names=[f'{model1_name} Pred', f'{model2_name} Pred']), fill_value=0)

        table_array_mcnemar = np.array([
            [contingency_table_mcnemar.loc[(0, 0)], contingency_table_mcnemar.loc[(0, 1)]],
            [contingency_table_mcnemar.loc[(1, 0)], contingency_table_mcnemar.loc[(1, 1)]]
        ])

        n01_scalar = int(table_array_mcnemar[0, 1])  # Model1 Pred 0, Model2 Pred 1
        n10_scalar = int(table_array_mcnemar[1, 0])  # Model1 Pred 1, Model2 Pred 0

        if n01_scalar + n10_scalar > 0:
            mcnemar_result = mcnemar(table_array_mcnemar, exact=True)
            ensemble_statistical_test_results[comparison_key]['McNemar p'] = mcnemar_result.pvalue
        else:
            ensemble_statistical_test_results[comparison_key]['McNemar p'] = 1.0


# Now, format the results into a readable table
print("\nStatistical Test Results Among Ensemble Learners:")

ensemble_comparison_data = []
for comparison, tests in ensemble_statistical_test_results.items():
    row = {'Comparison': comparison}
    for test_name, p_value in tests.items():
         if not np.isnan(p_value):
            row[test_name] = f"{p_value:.4f}"
            row[f'Significant? ({test_name})'] = '*' if p_value < alpha else ''
         else:
            row[test_name] = "N/A"
            row[f'Significant? ({test_name})'] = ''

    ensemble_comparison_data.append(row)

ensemble_comparison_df = pd.DataFrame(ensemble_comparison_data)

# Define a desired column order for clarity
ensemble_ordered_columns = ['Comparison']
for metric_name in ['ROC AUC', 'PR AUC', 'F1 Score']:
    ensemble_ordered_columns.append(f'{metric_name} (T-test p)')
    ensemble_ordered_columns.append(f'Significant? ({metric_name} (T-test p))')
    ensemble_ordered_columns.append(f'{metric_name} (Wilcoxon p)')
    ensemble_ordered_columns.append(f'Significant? ({metric_name} (Wilcoxon p))')
    ensemble_ordered_columns.append(f'{metric_name} (Mann-Whitney p)')
    ensemble_ordered_columns.append(f'Significant? ({metric_name} (Mann-Whitney p))')

ensemble_ordered_columns.append('McNemar p')
ensemble_ordered_columns.append('Significant? (McNemar p)')

# Filter columns to only include those present in the DataFrame
final_ensemble_columns = [col for col in ensemble_ordered_columns if col in ensemble_comparison_df.columns]
ensemble_comparison_df = ensemble_comparison_df[final_ensemble_columns]


print(ensemble_comparison_df.to_string())

# Optionally save this table
output_dir_tables = "/content/drive/MyDrive/Final_Implementations/DynaBiome_Tables/"
os.makedirs(output_dir_tables, exist_ok=True)
ensemble_comparison_df.to_csv(f"{output_dir_tables}/ensemble_statistical_comparison.csv", index=False)

# Prepare for LaTeX
latex_ensemble_df = ensemble_comparison_df.copy()
latex_ensemble_df.columns = latex_ensemble_df.columns.str.replace('_', '\\_')
latex_ensemble_df.columns = latex_ensemble_df.columns.str.replace('%', '\\%')
latex_ensemble_df.columns = latex_ensemble_df.columns.str.replace('(', '{(').str.replace(')', ')}')

latex_ensemble_df.to_latex(f"{output_dir_tables}/ensemble_statistical_comparison.tex", index=False, float_format="%.4f", escape=False)

print(f"\nEnsemble statistical comparison table saved to {output_dir_tables}/ensemble_statistical_comparison.csv and .tex")

# Statistical Tests Plots and Summary Table

In [ ]:
# Box/Violin plots of metrics
metric_names = ['ROC AUC', 'PR AUC', 'F1 Score']
for metric_name in metric_names:
    plt.figure(figsize=(12, 6))
    plot_data = []
    model_order = list(models.keys()) # Maintain a consistent order

    for model_name in model_order:
        if metric_name in metric_distributions[model_name] and metric_distributions[model_name][metric_name]:
             plot_data.append(pd.DataFrame({
                 'Model': model_name,
                 metric_name: metric_distributions[model_name][metric_name]
             }))


    if plot_data:
        combined_df = pd.concat(plot_data)
        # Ensure the order is maintained in the plot
        sns.boxplot(x='Model', y=metric_name, data=combined_df, palette='viridis', order=model_order)
        # sns.violinplot(x='Model', y=metric_name, data=combined_df, palette='viridis', inner="quartile", order=model_order) # Alternative violin plot
        plt.title(f'{metric_name} Distribution from Bootstrap ({n_iterations} Iterations)')
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()
        #plt.savefig(f"{output_dir_bootstrap_plots}/{metric_name.replace(' ', '_')}_Bootstrap_BoxPlot.pdf", dpi=600, bbox_inches='tight')
        plt.show()
    else:
        print(f"No data available to plot {metric_name}.")


# Function to plot ROC/PR curve with bootstrap confidence bands
def plot_curve_with_bands(model_name, bootstrap_samples_for_plotting, title_prefix, filename_prefix, curve_type='roc'):
    plt.figure(figsize=(10, 8))

    all_curves_interp = []
    n_bootstrap_success = 0 # Count how many bootstrap samples were successfully used for curves

    if curve_type == 'roc':
        base_fpr = np.linspace(0, 1, 101) # Standard x-axis for interpolation
        for y_true_sample, probs_sample in bootstrap_samples_for_plotting:

            if len(np.unique(y_true_sample)) > 1 and len(np.unique(probs_sample)) > 1:
                 fpr, tpr, _ = roc_curve(y_true_sample, probs_sample)
                 interp_tpr = np.interp(base_fpr, fpr, tpr)
                 interp_tpr[0] = 0.0 # Ensure start at (0,0)
                 all_curves_interp.append(interp_tpr)
                 n_bootstrap_success += 1

        if n_bootstrap_success > 1: # Need at least 2 curves to compute band
            mean_tpr = np.mean(all_curves_interp, axis=0)
            # Using t-distribution for smaller sample sizes, but for n=1000, Z is fine
            # std_tpr = np.std(all_curves_interp, axis=0)
            # ci_width = 1.96 * std_tpr / np.sqrt(n_bootstrap_success)
            # tprs_lower = mean_tpr - ci_width
            # tprs_upper = mean_tpr + ci_width

            # Percentile method for CI band - more robust to non-normal distributions
            tprs_lower = np.percentile(all_curves_interp, (alpha / 2) * 100, axis=0)
            tprs_upper = np.percentile(all_curves_interp, 100 - (alpha / 2) * 100, axis=0)


            tprs_lower = np.maximum(tprs_lower, 0)
            tprs_upper = np.minimum(tprs_upper, 1)

            # Mean AUC from the bootstrap distribution (calculated earlier)
            mean_auc_from_dist = np.mean(metric_distributions[model_name]['ROC AUC']) if metric_distributions[model_name]['ROC AUC'] else np.nan


            plt.plot(base_fpr, mean_tpr, label=f'Mean ROC (AUC = {mean_auc_from_dist:.3f})')
            plt.fill_between(base_fpr, tprs_lower, tprs_upper, color='grey', alpha=.2, label='95% CI')
            plt.plot([0, 1], [0, 1], 'k--', lw=1)
            plt.xlabel("False Positive Rate", fontsize=12)
            plt.ylabel("True Positive Rate", fontsize=12)
            plt.title(f'{title_prefix} ROC Curve with Bootstrap CI', fontsize=14)
            plt.legend(loc="lower right", fontsize=10)
        else:
             print(f"Not enough successful bootstrap iterations ({n_bootstrap_success}) to plot ROC CI band for {title_prefix}.")
             # Plot the curve on the original test set if band cannot be plotted
             # Check y_test directly for original plot
             if len(np.unique(y_test)) > 1 and len(np.unique(models[model_name]['probs'])) > 1:
                 fpr, tpr, _ = roc_curve(y_test, models[model_name]['probs'])
                 auc_orig = roc_auc_score(y_test, models[model_name]['probs'])
                 plt.plot(fpr, tpr, label=f'ROC (AUC = {auc_orig:.3f})')
                 plt.plot([0, 1], [0, 1], 'k--', lw=1)
                 plt.xlabel("False Positive Rate", fontsize=12)
                 plt.ylabel("True Positive Rate", fontsize=12)
                 plt.title(f'{title_prefix} ROC Curve (No CI band)', fontsize=14)
                 plt.legend(loc="lower right", fontsize=10)
             else:
                  print(f"Skipping ROC plot for {title_prefix} - not enough unique values in original data.")


    elif curve_type == 'pr':
         base_recall = np.linspace(0, 1, 101)
         all_precisions_interp = []
         n_bootstrap_success = 0

         for y_true_sample, probs_sample in bootstrap_samples_for_plotting:

            if len(np.unique(y_true_sample)) > 1 and len(np.unique(probs_sample)) > 1:
                precision, recall, _ = precision_recall_curve(y_true_sample, probs_sample)
                # Sort by recall for interpolation
                sort_indices = np.argsort(recall)
                recall = recall[sort_indices]
                precision = precision[sort_indices]

                # Handle interpolation carefully, especially at low recall
                # Append (0, 1) point if not present
                if recall[0] > 0:
                    recall = np.insert(recall, 0, 0)
                    precision = np.insert(precision, 0, 1)

                # Append (proportion of positives, proportion of positives) at recall 1 if not present
                # or ensure the last point is handled
                if recall[-1] < 1.0:
                     prop_pos = np.sum(y_true_sample) / len(y_true_sample) if len(y_true_sample) > 0 else 0
                     recall = np.append(recall, 1.0)
                     precision = np.append(precision, prop_pos) # Precision at recall 1 is proportion of positives

                interp_precision = np.interp(base_recall, recall, precision)

                all_precisions_interp.append(interp_precision)
                n_bootstrap_success += 1

         if n_bootstrap_success > 1: # Need at least 2 curves to compute band
            mean_precision = np.mean(all_precisions_interp, axis=0)
            # std_precision = np.std(all_precisions_interp, axis=0)
            # ci_width_pr = 1.96 * std_precision / np.sqrt(n_bootstrap_success)
            # precisions_lower = mean_precision - ci_width_pr
            # precisions_upper = mean_precision + ci_width_pr

            # Percentile method for CI band
            precisions_lower = np.percentile(all_precisions_interp, (alpha / 2) * 100, axis=0)
            precisions_upper = np.percentile(all_precisions_interp, 100 - (alpha / 2) * 100, axis=0)

            precisions_lower = np.maximum(precisions_lower, 0)
            precisions_upper = np.minimum(precisions_upper, 1)

            # Mean AP from the bootstrap distribution (calculated earlier)
            mean_ap_from_dist = np.mean(metric_distributions[model_name]['PR AUC']) if metric_distributions[model_name]['PR AUC'] else np.nan


            plt.plot(base_recall, mean_precision, label=f'Mean PR (AP = {mean_ap_from_dist:.3f})')
            plt.fill_between(base_recall, precisions_lower, precisions_upper, color='grey', alpha=.2, label='95% CI')
            plt.xlabel("Recall", fontsize=12)
            plt.ylabel("Precision", fontsize=12)
            plt.title(f'{title_prefix} PR Curve with Bootstrap CI', fontsize=14)
            plt.legend(loc="lower left", fontsize=10)
            plt.ylim([0.0, 1.05])
            plt.xlim([0.0, 1.0])
         else:
              print(f"Not enough successful bootstrap iterations ({n_bootstrap_success}) to plot PR CI band for {title_prefix}.")
              # Plot the curve on the original test set if band cannot be plotted
              if len(np.unique(y_test)) > 1 and len(np.unique(models[model_name]['probs'])) > 1:
                precision, recall, _ = precision_recall_curve(y_test, models[model_name]['probs'])
                ap_orig = average_precision_score(y_test, models[model_name]['probs'])
                plt.plot(recall, precision, label=f'PR (AP = {ap_orig:.3f})')
                plt.xlabel("Recall", fontsize=12)
                plt.ylabel("Precision", fontsize=12)
                plt.title(f'{title_prefix} PR Curve (No CI band)', fontsize=14)
                plt.legend(loc="lower left", fontsize=10)
                plt.ylim([0.0, 1.05])
                plt.xlim([0.0, 1.0])
              else:
                   print(f"Skipping PR plot for {title_prefix} - not enough unique values in original data.")


    plt.grid(True, linestyle='--', alpha=0.6)
    plt.tight_layout()
    #plt.savefig(f"{output_dir_bootstrap_plots}/{filename_prefix}_{curve_type.upper()}_Curve_Bootstrap_CI.pdf", dpi=600, bbox_inches='tight')
    plt.show()


# Plot curves with bands for each model
for model_name in models.keys():
     plot_curve_with_bands(
         model_name,
         metric_distributions[model_name]['bootstrap_samples_for_plotting'],
         model_name,
         model_name.replace(' ', '_').replace('-', '_'),
         curve_type='roc'
         )
     plot_curve_with_bands(
         model_name,
         metric_distributions[model_name]['bootstrap_samples_for_plotting'],
         model_name,
         model_name.replace(' ', '_').replace('-', '_'),
         curve_type='pr'
         )


print("\nGenerating Summary Table...")

summary_data = []
for model_name in models.keys():
    row: Dict[str, Any] = {'Model': model_name}

    for metric_name in metric_names:
        summary_info = metric_summary[model_name][metric_name]
        mean = summary_info['mean']
        ci_lower = summary_info['ci_lower']
        ci_upper = summary_info['ci_upper']

        if not np.isnan(mean):
             row[f'{metric_name} Mean ± 95% CI'] = f"{mean:.3f} ± ({mean - ci_lower:.3f}, {ci_upper - mean:.3f})"
        else:
             row[f'{metric_name} Mean ± 95% CI'] = "N/A"


    if model_name != 'LSTM-AE':
        vs_lstm_ae_tests = statistical_test_results['vs LSTM-AE'][model_name]
        for test_name, p_value in vs_lstm_ae_tests.items():
            if not np.isnan(p_value):
                row[f'{test_name} (vs LSTM-AE)'] = f"{p_value:.4f}"
                # Add significance flag (p < alpha)
                row[f'Significant? ({test_name})'] = '*' if p_value < alpha else ''
            else:
                 row[f'{test_name} (vs LSTM-AE)'] = "N/A"
                 row[f'Significant? ({test_name})'] = ''

    summary_data.append(row)

summary_df = pd.DataFrame(summary_data)

# Reorder columns for better readability
ordered_columns = ['Model']
for metric_name in metric_names:
    ordered_columns.append(f'{metric_name} Mean ± 95% CI')
    if metric_name != 'F1 Score': # Only ROC and PR have T-test and Wilcoxon on distribution
        ordered_columns.append(f'{metric_name} (T-test p) (vs LSTM-AE)')
        ordered_columns.append(f'Significant? ({metric_name} (T-test p))')
        ordered_columns.append(f'{metric_name} (Wilcoxon p) (vs LSTM-AE)')
        ordered_columns.append(f'Significant? ({metric_name} (Wilcoxon p))')
    else: # F1 score has T-test and Wilcoxon
        ordered_columns.append(f'{metric_name} (T-test p) (vs LSTM-AE)')
        ordered_columns.append(f'Significant? ({metric_name} (T-test p))')
        ordered_columns.append(f'{metric_name} (Wilcoxon p) (vs LSTM-AE)')
        ordered_columns.append(f'Significant? ({metric_name} (Wilcoxon p))')


ordered_columns.append('McNemar p (vs LSTM-AE)')
ordered_columns.append('Significant? (McNemar p)')

# Ensure only existing columns are included (handles LSTM-AE which has no vs tests)
final_columns = [col for col in ordered_columns if col in summary_df.columns]
summary_df = summary_df[final_columns]


print("\nSummary Table:")
print(summary_df.to_string()) # Use to_string to display the full table without truncation

# Save the summary table to a file
output_dir_tables = "/content/drive/MyDrive/Final_Implementations/DynaBiome_Tables/"
os.makedirs(output_dir_tables, exist_ok=True)
summary_df.to_csv(f"{output_dir_tables}/model_performance_summary.csv", index=False)
summary_df.to_latex(f"{output_dir_tables}/model_performance_summary.tex", index=False, float_format="%.3f")

print("\nAnalysis Complete.")

In [ ]:
# prompt: Suggestions for Improvement or Extension
# 1. Enhance Boxplot Aesthetics:
# Consider adding swarm/strip plots for better visualization of distribution density:
# sns.boxplot(...)  # Existing line
# sns.stripplot(x='Model', y=metric_name, data=combined_df, color='black', alpha=0.3, jitter=True, size=3)
# 2. Add Effect Sizes to Summary Table:
# P-values tell you if a difference exists, not how large it is.
# •	Include Cohen’s d or Cliff’s Delta for effect size on continuous metrics.
# •	For paired comparisons (e.g., ROC AUCs), Cohen’s d can be added easily:
# def cohen_d(x, y):
#     return (np.mean(x) - np.mean(y)) / np.sqrt((np.std(x, ddof=1) ** 2 + np.std(y, ddof=1) ** 2) / 2)
# 3. Show Sample Size in Plots:
# Add bootstrap sample count directly in the titles:
# plt.title(f'{title_prefix} ROC Curve with Bootstrap CI\n(N = {n_bootstrap_success} bootstrap samples)', fontsize=14)
# 4. Make Alpha Adjustable:
# Instead of a hardcoded alpha, you could parameterize it:
# def plot_curve_with_bands(..., alpha=0.05):
# 5. Save Plots in Multiple Formats (PDF, PNG):
# To ensure compatibility across journals and presentations:
# plt.savefig(f"{output_dir}/{filename}.pdf", dpi=600)
# plt.savefig(f"{output_dir}/{filename}.png", dpi=300)
# Optional: Parallelize Bootstrap Computation
# If bootstrap iteration is time-consuming, you could parallelize curve computation using joblib:
# from joblib import Parallel, delayed
# def compute_bootstrap_curve(...):  # Wrapper function
#     ...
# results = Parallel(n_jobs=-1)(delayed(compute_bootstrap_curve)(...) for _ in range(n_iterations))
# Optional: Include Metrics Distribution in Summary Table
# Add SD or full distribution stats:
# row[f'{metric_name} Std'] = f"{np.std(metric_distributions[model_name][metric_name]):.3f}"

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Enhance Boxplot Aesthetics: Add swarm/strip plots
metric_names = ['ROC AUC', 'PR AUC', 'F1 Score']
output_dir_bootstrap_plots = "/content/drive/MyDrive/Final_Implementations/DynaBiome_Figures/Bootstrap_Plots/"
os.makedirs(output_dir_bootstrap_plots, exist_ok=True)

for metric_name in metric_names:
    plt.figure(figsize=(12, 6))
    plot_data = []
    model_order = list(models.keys())  # Maintain a consistent order

    for model_name in model_order:
        if metric_name in metric_distributions[model_name] and metric_distributions[model_name][metric_name]:
            plot_data.append(pd.DataFrame({
                'Model': model_name,
                metric_name: metric_distributions[model_name][metric_name]
            }))

    if plot_data:
        combined_df = pd.concat(plot_data)
        # Ensure the order is maintained in the plot
        sns.boxplot(x='Model', y=metric_name, data=combined_df, palette='viridis', order=model_order)
        # Add stripplot for individual data points
        sns.stripplot(x='Model', y=metric_name, data=combined_df, color='black', alpha=0.3, jitter=True, size=3, order=model_order)

        plt.title(f'{metric_name} Distribution from Bootstrap ({n_iterations} Iterations)')
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()
        plt.savefig(f"{output_dir_bootstrap_plots}/{metric_name.replace(' ', '_')}_Bootstrap_BoxPlot_Stripplot.pdf", dpi=600, bbox_inches='tight')
        plt.show()
    else:
        print(f"No data available to plot {metric_name}.")


# Modify plot_curve_with_bands to show sample size in title
def plot_curve_with_bands(model_name, bootstrap_samples_for_plotting, title_prefix, filename_prefix, curve_type='roc', n_bootstrap_total=None):
    plt.figure(figsize=(10, 8))

    all_curves_interp = []
    n_bootstrap_success = 0  # Count how many bootstrap samples were successfully used for curves

    if curve_type == 'roc':
        base_fpr = np.linspace(0, 1, 101)  # Standard x-axis for interpolation
        for y_true_sample, probs_sample in bootstrap_samples_for_plotting:
            if len(np.unique(y_true_sample)) > 1 and len(np.unique(probs_sample)) > 1:
                fpr, tpr, _ = roc_curve(y_true_sample, probs_sample)
                interp_tpr = np.interp(base_fpr, fpr, tpr)
                interp_tpr[0] = 0.0  # Ensure start at (0,0)
                all_curves_interp.append(interp_tpr)
                n_bootstrap_success += 1

        if n_bootstrap_success > 1:  # Need at least 2 curves to compute band
            mean_tpr = np.mean(all_curves_interp, axis=0)
            tprs_lower = np.percentile(all_curves_interp, (alpha / 2) * 100, axis=0)
            tprs_upper = np.percentile(all_curves_interp, 100 - (alpha / 2) * 100, axis=0)

            tprs_lower = np.maximum(tprs_lower, 0)
            tprs_upper = np.minimum(tprs_upper, 1)

            mean_auc_from_dist = np.mean(metric_distributions[model_name]['ROC AUC']) if metric_distributions[model_name]['ROC AUC'] else np.nan

            plt.plot(base_fpr, mean_tpr, label=f'Mean ROC (AUC = {mean_auc_from_dist:.3f})')
            plt.fill_between(base_fpr, tprs_lower, tprs_upper, color='grey', alpha=.2, label='95% CI')
            plt.plot([0, 1], [0, 1], 'k--', lw=1)
            plt.xlabel("False Positive Rate", fontsize=12)
            plt.ylabel("True Positive Rate", fontsize=12)
            # Include sample size in title
            plt.title(f'{title_prefix} ROC Curve with Bootstrap CI\n(N = {n_bootstrap_total} bootstrap samples, {n_bootstrap_success} successful)', fontsize=14)
            plt.legend(loc="lower right", fontsize=10)
        else:
            print(f"Not enough successful bootstrap iterations ({n_bootstrap_success}) to plot ROC CI band for {title_prefix}.")
            # Plot the curve on the original test set if band cannot be plotted
            if len(np.unique(y_test)) > 1 and len(np.unique(models[model_name]['probs'])) > 1:
                fpr, tpr, _ = roc_curve(y_test, models[model_name]['probs'])
                auc_orig = roc_auc_score(y_test, models[model_name]['probs'])
                plt.plot(fpr, tpr, label=f'ROC (AUC = {auc_orig:.3f})')
                plt.plot([0, 1], [0, 1], 'k--', lw=1)
                plt.xlabel("False Positive Rate", fontsize=12)
                plt.ylabel("True Positive Rate", fontsize=12)
                plt.title(f'{title_prefix} ROC Curve (No CI band)\n(N = {len(y_test)} test samples)', fontsize=14) # Use test set size for this case
                plt.legend(loc="lower right", fontsize=10)
            else:
                print(f"Skipping ROC plot for {title_prefix} - not enough unique values in original data.")


    elif curve_type == 'pr':
        base_recall = np.linspace(0, 1, 101)
        all_precisions_interp = []
        n_bootstrap_success = 0

        for y_true_sample, probs_sample in bootstrap_samples_for_plotting:
            if len(np.unique(y_true_sample)) > 1 and len(np.unique(probs_sample)) > 1:
                precision, recall, _ = precision_recall_curve(y_true_sample, probs_sample)
                sort_indices = np.argsort(recall)
                recall = recall[sort_indices]
                precision = precision[sort_indices]

                if recall[0] > 0:
                    recall = np.insert(recall, 0, 0)
                    precision = np.insert(precision, 0, 1)

                if recall[-1] < 1.0:
                    prop_pos = np.sum(y_true_sample) / len(y_true_sample) if len(y_true_sample) > 0 else 0
                    recall = np.append(recall, 1.0)
                    precision = np.append(precision, prop_pos)

                interp_precision = np.interp(base_recall, recall, precision)

                all_precisions_interp.append(interp_precision)
                n_bootstrap_success += 1

        if n_bootstrap_success > 1:  # Need at least 2 curves to compute band
            mean_precision = np.mean(all_precisions_interp, axis=0)
            precisions_lower = np.percentile(all_precisions_interp, (alpha / 2) * 100, axis=0)
            precisions_upper = np.percentile(all_precisions_interp, 100 - (alpha / 2) * 100, axis=0)

            precisions_lower = np.maximum(precisions_lower, 0)
            precisions_upper = np.minimum(precisions_upper, 1)

            mean_ap_from_dist = np.mean(metric_distributions[model_name]['PR AUC']) if metric_distributions[model_name]['PR AUC'] else np.nan

            plt.plot(base_recall, mean_precision, label=f'Mean PR (AP = {mean_ap_from_dist:.3f})')
            plt.fill_between(base_recall, precisions_lower, precisions_upper, color='grey', alpha=.2, label='95% CI')
            plt.xlabel("Recall", fontsize=12)
            plt.ylabel("Precision", fontsize=12)
            # Include sample size in title
            plt.title(f'{title_prefix} PR Curve with Bootstrap CI\n(N = {n_bootstrap_total} bootstrap samples, {n_bootstrap_success} successful)', fontsize=14)
            plt.legend(loc="lower left", fontsize=10)
            plt.ylim([0.0, 1.05])
            plt.xlim([0.0, 1.0])
        else:
            print(f"Not enough successful bootstrap iterations ({n_bootstrap_success}) to plot PR CI band for {title_prefix}.")
            # Plot the curve on the original test set if band cannot be plotted
            if len(np.unique(y_test)) > 1 and len(np.unique(models[model_name]['probs'])) > 1:
                precision, recall, _ = precision_recall_curve(y_test, models[model_name]['probs'])
                ap_orig = average_precision_score(y_test, models[model_name]['probs'])
                plt.plot(recall, precision, label=f'PR (AP = {ap_orig:.3f})')
                plt.xlabel("Recall", fontsize=12)
                plt.ylabel("Precision", fontsize=12)
                plt.title(f'{title_prefix} PR Curve (No CI band)\n(N = {len(y_test)} test samples)', fontsize=14) # Use test set size for this case
                plt.legend(loc="lower left", fontsize=10)
                plt.ylim([0.0, 1.05])
                plt.xlim([0.0, 1.0])
            else:
                print(f"Skipping PR plot for {title_prefix} - not enough unique values in original data.")

    plt.grid(True, linestyle='--', alpha=0.6)
    plt.tight_layout()
    plt.savefig(f"{output_dir_bootstrap_plots}/{filename_prefix}_{curve_type.upper()}_Curve_Bootstrap_CI.pdf", dpi=600, bbox_inches='tight')
    plt.show()

# Plot curves with bands using the modified function
for model_name in models.keys():
    plot_curve_with_bands(
        model_name,
        metric_distributions[model_name]['bootstrap_samples_for_plotting'],
        model_name,
        model_name.replace(' ', '_').replace('-', '_'),
        curve_type='roc',
        n_bootstrap_total=n_iterations # Pass the total number of iterations
    )
    plot_curve_with_bands(
        model_name,
        metric_distributions[model_name]['bootstrap_samples_for_plotting'],
        model_name,
        model_name.replace(' ', '_').replace('-', '_'),
        curve_type='pr',
        n_bootstrap_total=n_iterations # Pass the total number of iterations
    )

# Add Effect Sizes to Summary Table: Include Cohen’s d
# Cohen's d for paired samples (appropriate for paired bootstrap samples)
# We'll compare each model's bootstrap distribution to the LSTM-AE's distribution

def cohen_d_paired(x, y):
    """Calculates Cohen's d for paired samples."""
    diff = np.array(x) - np.array(y)
    mean_diff = np.mean(diff)
    std_diff = np.std(diff, ddof=1) # Use ddof=1 for sample standard deviation
    # Handle case where std_diff is zero
    if std_diff == 0:
        return np.inf if mean_diff != 0 else 0.0
    return mean_diff / std_diff

print("\nCalculating Effect Sizes (Cohen's d vs LSTM-AE) and updating Summary Table...")

# Update summary_data list with effect sizes
summary_data = []
for model_name in models.keys():
    row: Dict[str, Any] = {'Model': model_name}

    for metric_name in metric_names:
        summary_info = metric_summary[model_name][metric_name]
        mean = summary_info['mean']
        ci_lower = summary_info['ci_lower']
        ci_upper = summary_info['ci_upper']
        distribution = summary_info['distribution'] # Get the distribution

        if not np.isnan(mean):
            row[f'{metric_name} Mean (95% CI)'] = f"{mean:.3f} ({ci_lower:.3f} - {ci_upper:.3f})"
        else:
            row[f'{metric_name} Mean (95% CI)'] = "N/A"

        # Calculate Cohen's d vs LSTM-AE if not the LSTM-AE model and distributions exist
        if model_name != 'LSTM-AE' and distribution and metric_distributions['LSTM-AE'][metric_name]:
             lstm_ae_dist = metric_distributions['LSTM-AE'][metric_name]
             min_len = min(len(distribution), len(lstm_ae_dist))

             if min_len >= 2: # Need at least 2 samples for Cohen's d
                # Use the truncated distributions for Cohen's d to match the paired test length
                d = cohen_d_paired(distribution[:min_len], lstm_ae_dist[:min_len])
                row[f'{metric_name} Cohen\'s d (vs LSTM-AE)'] = f"{d:.3f}"
             else:
                 row[f'{metric_name} Cohen\'s d (vs LSTM-AE)'] = "N/A"
        else:
            row[f'{metric_name} Cohen\'s d (vs LSTM-AE)'] = "N/A"


    if model_name != 'LSTM-AE':
        vs_lstm_ae_tests = statistical_test_results['vs LSTM-AE'][model_name]
        for test_name, p_value in vs_lstm_ae_tests.items():
            if not np.isnan(p_value):
                row[f'{test_name} (vs LSTM-AE)'] = f"{p_value:.4f}"
                row[f'Sig ({test_name})'] = '*' if p_value < alpha else ''
            else:
                row[f'{test_name} (vs LSTM-AE)'] = "N/A"
                row[f'Sig ({test_name})'] = ''

    summary_data.append(row)

summary_df_updated = pd.DataFrame(summary_data)

# Reorder columns for better readability with effect sizes
ordered_columns_updated = ['Model']
for metric_name in metric_names:
    ordered_columns_updated.append(f'{metric_name} Mean (95% CI)')
    ordered_columns_updated.append(f'{metric_name} Cohen\'s d (vs LSTM-AE)')
    # Add p-values and significance flags for statistical tests related to this metric
    if metric_name != 'F1 Score':
         ordered_columns_updated.append(f'{metric_name} (T-test p) (vs LSTM-AE)')
         ordered_columns_updated.append(f'Sig ({metric_name} (T-test p))')
         ordered_columns_updated.append(f'{metric_name} (Wilcoxon p) (vs LSTM-AE)')
         ordered_columns_updated.append(f'Sig ({metric_name} (Wilcoxon p))')
         ordered_columns_updated.append(f'{metric_name} (Mann-Whitney p) (vs LSTM-AE)')
         ordered_columns_updated.append(f'Sig ({metric_name} (Mann-Whitney p))')

    else:
        ordered_columns_updated.append(f'{metric_name} (T-test p) (vs LSTM-AE)')
        ordered_columns_updated.append(f'Sig ({metric_name} (T-test p))')
        ordered_columns_updated.append(f'{metric_name} (Wilcoxon p) (vs LSTM-AE)')
        ordered_columns_updated.append(f'Sig ({metric_name} (Wilcoxon p))')
        ordered_columns_updated.append(f'{metric_name} (Mann-Whitney p) (vs LSTM-AE)')
        ordered_columns_updated.append(f'Sig ({metric_name} (Mann-Whitney p))')


# Add McNemar test results at the end
ordered_columns_updated.append('McNemar p (vs LSTM-AE)')
ordered_columns_updated.append('Sig (McNemar p)')


# Ensure only existing columns are included
final_columns_updated = [col for col in ordered_columns_updated if col in summary_df_updated.columns]
summary_df_updated = summary_df_updated[final_columns_updated]


print("\nUpdated Summary Table with Effect Sizes and Detailed Statistical Tests:")
print(summary_df_updated.to_string())

# Save the updated summary table
summary_df_updated.to_csv(f"{output_dir_tables}/model_performance_summary_with_effect_sizes.csv", index=False)
# Escape underscores for LaTeX
summary_df_updated.columns = summary_df_updated.columns.str.replace('_', '\\_')
summary_df_updated.to_latex(f"{output_dir_tables}/model_performance_summary_with_effect_sizes.tex", index=False, float_format="%.3f", escape=False)

In [ ]:
import pandas as pd
import numpy as np
# Generate the summary table from the collected data
summary_data = []
for model_name in models.keys():
    row = {'Model': model_name}
    # Add Mean and 95% CI for each metric
    for metric_name in metric_names:
        summary_info = metric_summary[model_name][metric_name]
        mean = summary_info['mean']
        ci_lower = summary_info['ci_lower']
        ci_upper = summary_info['ci_upper']

        if not np.isnan(mean):
            # Format as "Mean (Lower CI - Upper CI)" for clarity
            row[f'{metric_name} Mean (95% CI)'] = f"{mean:.3f} ({ci_lower:.3f} - {ci_upper:.3f})"
        else:
            row[f'{metric_name} Mean (95% CI)'] = "N/A"

    # Add Cohen's d vs LSTM-AE
    for metric_name in metric_names:
        cohen_d_key = f'{metric_name} Cohen\'s d (vs LSTM-AE)'
        if cohen_d_key in summary_df_updated.columns: # Check if the column exists (will not exist for LSTM-AE row)
            cohen_d_value = summary_df_updated.loc[summary_df_updated['Model'] == model_name, cohen_d_key].iloc[0]
            row[cohen_d_key] = cohen_d_value
        else:
             row[cohen_d_key] = "N/A"


    # Add Statistical Test p-values and Significance flags vs LSTM-AE
    if model_name != 'LSTM-AE':
        vs_lstm_ae_tests = statistical_test_results['vs LSTM-AE'][model_name]
        for test_name, p_value in vs_lstm_ae_tests.items():
            # Extract base metric name and test type
            parts = test_name.rsplit(' ', 3) # Split from right 3 times
            if len(parts) == 4: # Expected format "Metric Name (Test Type p)"
                 metric_base_name = parts[0]
                 test_type_part = f'{parts[1]} {parts[2]}' # e.g., "(T-test p)" or "(Wilcoxon p)"
                 test_column_name = f'{metric_base_name} {test_type_part} (vs LSTM-AE)'
                 sig_column_name = f'Sig ({metric_base_name} {test_type_part})'

                 if test_column_name in summary_df_updated.columns:
                    row[test_column_name] = summary_df_updated.loc[summary_df_updated['Model'] == model_name, test_column_name].iloc[0]
                    row[sig_column_name] = summary_df_updated.loc[summary_df_updated['Model'] == model_name, sig_column_name].iloc[0]
            elif test_name.startswith('McNemar'): # Special case for McNemar
                test_column_name = 'McNemar p (vs LSTM-AE)'
                sig_column_name = 'Sig (McNemar p)'
                if test_column_name in summary_df_updated.columns:
                    row[test_column_name] = summary_df_updated.loc[summary_df_updated['Model'] == model_name, test_column_name].iloc[0]
                    row[sig_column_name] = summary_df_updated.loc[summary_df_updated['Model'] == model_name, sig_column_name].iloc[0]


    summary_data.append(row)


final_summary_df = pd.DataFrame(summary_data)

# Reorder columns for the final table structure
# Ensure all potential columns from the previous calculations are included
potential_columns = ['Model']
for metric_name in metric_names:
    potential_columns.append(f'{metric_name} Mean (95% CI)')
    potential_columns.append(f'{metric_name} Cohen\'s d (vs LSTM-AE)')
    potential_columns.append(f'{metric_name} (T-test p) (vs LSTM-AE)')
    potential_columns.append(f'Sig ({metric_name} (T-test p))')
    potential_columns.append(f'{metric_name} (Wilcoxon p) (vs LSTM-AE)')
    potential_columns.append(f'Sig ({metric_name} (Wilcoxon p))')
    potential_columns.append(f'{metric_name} (Mann-Whitney p) (vs LSTM-AE)')
    potential_columns.append(f'Sig ({metric_name} (Mann-Whitney p))') # Ensure Mann-Whitney is included

potential_columns.append('McNemar p (vs LSTM-AE)')
potential_columns.append('Sig (McNemar p)')


# Filter columns to only include those present in the final_summary_df
final_columns_ordered = [col for col in potential_columns if col in final_summary_df.columns]
final_summary_df = final_summary_df[final_columns_ordered]


print("\nFinal Comprehensive Summary Table:")
# Display the table, preventing truncation
print(final_summary_df.to_string())

# Save the final comprehensive table
final_summary_df.to_csv(f"{output_dir_tables}/final_model_performance_summary.csv", index=False)

# Prepare for LaTeX: Escape special characters, especially underscores in column names
latex_df = final_summary_df.copy()
latex_df.columns = latex_df.columns.str.replace('_', '\\_')
latex_df.columns = latex_df.columns.str.replace('%', '\\%') # Escape % as well
latex_df.columns = latex_df.columns.str.replace('(', '{(').str.replace(')', ')}') # Handle parentheses in column names
latex_df.to_latex(f"{output_dir_tables}/final_model_performance_summary.tex", index=False, float_format="%.3f", escape=False)

print(f"\nSummary table saved to {output_dir_tables}/final_model_performance_summary.csv and .tex")


# Summary Table

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score, average_precision_score, f1_score
from typing import Dict, List, Any
from scipy.stats import ttest_rel, wilcoxon, mannwhitneyu
from statsmodels.stats.contingency_tables import mcnemar # Correct import for mcnemar

# Assuming models, metric_summary, and statistical_test_results dictionaries
# are populated from the previous code execution.
# Also assuming metric_distributions is available and contains the bootstrap distributions.

# Define the metrics we want in the table
metric_names = ['ROC AUC', 'PR AUC', 'F1 Score']

print("\nGenerating Comprehensive Summary Table...")

summary_data = []
for model_name in models.keys():
    row: Dict[str, Any] = {'Model': model_name}

    # Add Mean and 95% CI for each metric
    for metric_name in metric_names:
        # Ensure the metric exists for the model in metric_summary
        if model_name in metric_summary and metric_name in metric_summary[model_name]:
            summary_info = metric_summary[model_name][metric_name]
            mean = summary_info['mean']
            ci_lower = summary_info['ci_lower']
            ci_upper = summary_info['ci_upper']

            if not np.isnan(mean):
                # Format as "Mean (Lower CI - Upper CI)"
                row[f'{metric_name} Mean (95% CI)'] = f"{mean:.3f} ({ci_lower:.3f} - {ci_upper:.3f})"
            else:
                row[f'{metric_name} Mean (95% CI)'] = "N/A"
        else:
            row[f'{metric_name} Mean (95% CI)'] = "N/A" # Metric info missing


    # Add Statistical Test p-values and Significance flags vs LSTM-AE
    if model_name != 'LSTM-AE':
        # Ensure test results exist for the model
        if 'vs LSTM-AE' in statistical_test_results and model_name in statistical_test_results['vs LSTM-AE']:
            vs_lstm_ae_tests = statistical_test_results['vs LSTM-AE'][model_name]

            # Define the specific tests to include and their order
            tests_to_include = {
                'ROC AUC (T-test p)': 'ROC AUC (T-test p) (vs LSTM-AE)',
                'ROC AUC (Wilcoxon p)': 'ROC AUC (Wilcoxon p) (vs LSTM-AE)',
                # 'ROC AUC (Mann-Whitney p)': 'ROC AUC (Mann-Whitney p) (vs LSTM-AE)', # Include if Mann-Whitney was used
                'PR AUC (T-test p)': 'PR AUC (T-test p) (vs LSTM-AE)',
                'PR AUC (Wilcoxon p)': 'PR AUC (Wilcoxon p) (vs LSTM-AE)',
                 # 'PR AUC (Mann-Whitney p)': 'PR AUC (Mann-Whitney p) (vs LSTM-AE)', # Include if Mann-Whitney was used
                'F1 Score (T-test p)': 'F1 Score (T-test p) (vs LSTM-AE)',
                'F1 Score (Wilcoxon p)': 'F1 Score (Wilcoxon p) (vs LSTM-AE)',
                # 'F1 Score (Mann-Whitney p)': 'F1 Score (Mann-Whitney p) (vs LSTM-AE)', # Include if Mann-Whitney was used
                'McNemar p': 'McNemar p (vs LSTM-AE)'
            }

            for test_key, column_name in tests_to_include.items():
                p_value = vs_lstm_ae_tests.get(test_key) # Use .get to handle potentially missing keys

                if p_value is not None and not np.isnan(p_value):
                    row[column_name] = f"{p_value:.4f}"
                    # Add significance flag (p < 0.05)
                    row[f'Significant? ({test_key})'] = '*' if p_value < 0.05 else ''
                else:
                    row[column_name] = "N/A"
                    row[f'Significant? ({test_key})'] = ''
        else:
            # Add N/A for all test columns if no test results are found for the model
            # This ensures consistent columns even if testing failed for a model
            tests_to_include = {
                'ROC AUC (T-test p)': 'ROC AUC (T-test p) (vs LSTM-AE)',
                'ROC AUC (Wilcoxon p)': 'ROC AUC (Wilcoxon p) (vs LSTM-AE)',
                # 'ROC AUC (Mann-Whitney p)': 'ROC AUC (Mann-Whitney p) (vs LSTM-AE)',
                'PR AUC (T-test p)': 'PR AUC (T-test p) (vs LSTM-AE)',
                'PR AUC (Wilcoxon p)': 'PR AUC (Wilcoxon p) (vs LSTM-AE)',
                # 'PR AUC (Mann-Whitney p)': 'PR AUC (Mann-Whitney p) (vs LSTM-AE)',
                'F1 Score (T-test p)': 'F1 Score (T-test p) (vs LSTM-AE)',
                'F1 Score (Wilcoxon p)': 'F1 Score (Wilcoxon p) (vs LSTM-AE)',
                # 'F1 Score (Mann-Whitney p)': 'F1 Score (Mann-Whitney p) (vs LSTM-AE)',
                'McNemar p': 'McNemar p (vs LSTM-AE)'
            }
            for test_key, column_name in tests_to_include.items():
                row[column_name] = "N/A"
                row[f'Significant? ({test_key})'] = ''


    summary_data.append(row)

# Create the DataFrame from the collected data
summary_df = pd.DataFrame(summary_data)

# Define the desired column order
# This ensures the table has a consistent structure
ordered_columns = ['Model']
for metric_name in metric_names:
    ordered_columns.append(f'{metric_name} Mean (95% CI)')
    # Add test columns related to this metric
    if metric_name == 'ROC AUC':
        ordered_columns.append('ROC AUC (T-test p) (vs LSTM-AE)')
        ordered_columns.append('Significant? (ROC AUC (T-test p))')
        ordered_columns.append('ROC AUC (Wilcoxon p) (vs LSTM-AE)')
        ordered_columns.append('Significant? (ROC AUC (Wilcoxon p))')
        # ordered_columns.append('ROC AUC (Mann-Whitney p) (vs LSTM-AE)') # Include if used
        # ordered_columns.append('Significant? (ROC AUC (Mann-Whitney p))')
    elif metric_name == 'PR AUC':
        ordered_columns.append('PR AUC (T-test p) (vs LSTM-AE)')
        ordered_columns.append('Significant? (PR AUC (T-test p))')
        ordered_columns.append('PR AUC (Wilcoxon p) (vs LSTM-AE)')
        ordered_columns.append('Significant? (PR AUC (Wilcoxon p))')
        # ordered_columns.append('PR AUC (Mann-Whitney p) (vs LSTM-AE)') # Include if used
        # ordered_columns.append('Significant? (PR AUC (Mann-Whitney p))')
    elif metric_name == 'F1 Score':
        ordered_columns.append('F1 Score (T-test p) (vs LSTM-AE)')
        ordered_columns.append('Significant? (F1 Score (T-test p))')
        ordered_columns.append('F1 Score (Wilcoxon p) (vs LSTM-AE)')
        ordered_columns.append('Significant? (F1 Score (Wilcoxon p))')
        # ordered_columns.append('F1 Score (Mann-Whitney p) (vs LSTM-AE)') # Include if used
        # ordered_columns.append('Significant? (F1 Score (Mann-Whitney p))')


# Add McNemar's test results
ordered_columns.append('McNemar p (vs LSTM-AE)')
ordered_columns.append('Significant? (McNemar p)')


# Filter columns to only include those that actually exist in the DataFrame
final_columns = [col for col in ordered_columns if col in summary_df.columns]
summary_df = summary_df[final_columns]


print("\nSummary Table:")
# Use to_string() to print the entire DataFrame without truncation
print(summary_df.to_string())

# You might want to save this DataFrame to a file (e.g., CSV, Excel)
# Example: Save to CSV
# summary_df.to_csv("model_performance_summary.csv", index=False)

In [ ]:
#pip install -r requirements.txt

<a name="usage"></a>
## Usage

To use this project, follow these steps:

1.  **Setup**: Ensure you have the necessary dependencies installed as described in the [Setup](#setup) section.
2.  **Data**: Place your `asv_interpretability_dataset_modified.csv` file in the `/content/` directory or update the data loading code to point to your data location.
3.  **Run the Notebook**: Execute the code cells in the notebook sequentially. The notebook is designed to perform the data preprocessing, model training, evaluation, and interpretation steps.
4.  **Analyze Results**: Review the generated plots and the summary table to understand the model performance and feature importance. The output files (PDFs and CSVs) will be saved in the specified directories.
5.  **Adapt the Code**: Modify the code as needed for your specific dataset or research questions.

<a name="license"></a>
## License

In [ ]:
                                 Apache License
                           Version 2.0, January 2004
                        http://www.apache.org/licenses/

   TERMS AND CONDITIONS FOR USE, REPRODUCTION, AND DISTRIBUTION

   1. Definitions.

      "License" shall mean the terms and conditions for use, reproduction,
      and distribution as defined by Sections 1 through 9 of this document.

      "Licensor" shall mean the copyright owner or entity authorized by
      the copyright owner that is granting the License.

      "Legal Entity" shall mean the union of the acting entity and all
      other entities that control, are controlled by, or are under common
      control with that entity. For the purposes of this definition,
      "control" means (i) the power, direct or indirect, to cause the
      direction or management of such entity, whether by contract or
      otherwise, or (ii) ownership of fifty percent (50%) or more of the
      outstanding shares, or (iii) beneficial ownership of such entity.

      "You" (or "Your") shall mean an individual or Legal Entity
      exercising permissions granted by this License.

      "Source" form shall mean the preferred form for making modifications,
      including but not limited to software source code, documentation
      source, and configuration files.

      "Object" form shall mean any form resulting from mechanical
      transformation or translation of a Source form, including but
      not limited to compiled object code, generated documentation,
      and conversions to other media types.

      "Contribution" shall mean any work of authorship, including
      the original version of the Work and any modifications or additions
      to that Work or Derivative Works thereof, that is intentionally
      submitted to Licensor for inclusion in the Work by the copyright owner
      or by an individual or Legal Entity authorized to submit on behalf of
      the copyright owner. For the purposes of this definition, "submitted"
      means any form of electronic, verbal, or written communication sent
      to the Licensor or its representatives, including but not limited to
      communication on electronic mailing lists, source code control systems,
      and issue tracking systems that are managed by, or on behalf of, the
      Licensor for the purpose of discussing and improving the Work, but
      excluding communication that is conspicuously marked or otherwise
      designated in writing by the copyright owner as "Not a Contribution."

      "Contributor" shall mean any individual or Legal Entity on behalf of
      whom a Contribution has been received by Licensor and subsequently
      accepted by Licensor.

      "Derivative Works" shall mean any work, whether in Source or Object
      form, that is based on (or derived from) the Work and for which the
      editorial revisions, annotations, elaborations, or other modifications
      represent, as a whole, an original work of authorship. For this
      definition, Derivative Works shall not include works that remain
      separable from, or merely link (or bind by name) to the interfaces of
      the Work and Derivative Works thereof.

      "Distribution" shall mean any means by which You provide, make available,
      or otherwise transfer copies of the Work.

      "Submit" means any act of handing in, sending, or transmitting
      a Contribution to the Licensor or its representatives.

      "Contributor Grant" shall mean the Patent Grant and Copyright Grant
      granted by Contributors to Licensor.

      "Third Party" shall mean any individual or entity other than, or in
      addition to, Licensor and You.

      "Work" shall mean the work of authorship, whether in Source or Object
      form, made available under the License, as indicated by a copyright
      notice that is included in or attached to the work (an example is
      provided in the Appendix below).

   2. Grant of Copyright License. Subject to the terms and conditions of
      this License, each Contributor hereby grants to You a perpetual,
      worldwide, non-exclusive, no-charge, royalty-free, irrevocable
      copyright license to reproduce, prepare Derivative Works of,
      publicly display, publicly perform, sublicense, and distribute the
      Work and such Derivative Works in Source or Object form.

   3. Grant of Patent License. Subject to the terms and conditions of
      this License, each Contributor hereby grants to You a perpetual,
      worldwide, non-exclusive, no-charge, royalty-free, irrevocable
      (except as stated in this section) patent license to make, have made,
      use, offer to sell, sell, import, and otherwise transfer the Work,
      where such license applies only to those patent claims licensed by
      each Contributor that are necessarily infringed by their Contribution(s)
      alone or by combination of their Contribution(s) with the Work to which
      such Contribution(s) was submitted. If You institute patent litigation
      against any entity (including a cross-claim or counterclaim in a lawsuit)
      alleging that the Work or a Contribution incorporated within the Work
      constitutes direct or contributory patent infringement, then any patent
      licenses granted to You under this License for that Work shall terminate
      as of the date such litigation is filed.

   4. Redistribution. You may reproduce and distribute copies of the Work or
      Derivative Works thereof in any medium, with or without modifications,
      and in Source or Object form, provided that You meet the following
      conditions:

      (a) You must give any other recipients of the Work or Derivative Works
          a copy of this License; and

      (b) You must cause any modified files to carry prominent notices stating
          that You changed the files; and

      (c) You must retain, in the Source form of any Derivative Works that
          You distribute, all copyright, patent, trademark, and attribution
          notices from the Source form of the Work, excluding those notices that
          do not pertain to any part of the Derivative Works; and

      (d) If the Work includes a "NOTICE" text file as part of its
          Distribution, then any Derivative Works that You distribute must
          include a readable copy of the attribution notices contained
          within such NOTICE file, excluding those notices that do not
          pertain to any part of the Derivative Works, in at least one
          of the following places: within a NOTICE text file distributed
          as part of the Derivative Works; within the Source form or
          documentation, if provided along with the Derivative Works; or,
          within a display generated by the Derivative Works, if and
          wherever such third-party notices normally appear. The contents
          of the NOTICE file are for informational purposes only and do
          not modify the License. You may add Your own attribution notices
          within Derivative Works that You distribute, alongside, or as
          an addendum to the NOTICE text from the Work, provided that
          such additional attribution notices cannot be construed as
          modifying the License.

      You may add Your own copyright statement to Your modifications and may
      provide additional or different license terms and conditions for use,
      reproduction, or distribution of Your modifications, or for any
      such Derivative Works as a whole, provided Your use, reproduction,
      and distribution of the Work otherwise complies with the conditions stated in this License.

   5. Submission of Contributions. Unless You explicitly state otherwise,
      any Contribution intentionally submitted for inclusion in the Work by
      You to the Licensor shall be under the terms and conditions of this
      License, without any additional terms or conditions.
      Notwithstanding the above, nothing herein shall supersede or modify
      the terms of any separate license agreement you may have executed
      with Licensor regarding such Contributions.

   6. Trademarks. This License does not grant permission to use the trade
      names, trademarks, service marks, or product names of the Licensor,
      except as required for reasonable and customary usage in describing the
      origin of the Work and reproducing the content of the NOTICE file.

   7. Disclaimer of Warranty. Unless required by applicable law or
      agreed to in writing, Licensor provides the Work (and each Contributor
      provides its Contributions) on an "AS IS" BASIS, WITHOUT WARRANTIES OR
      CONDITIONS OF ANY KIND, either express or implied, including, without
      limitation, any warranties or conditions of TITLE, NON-INFRINGEMENT,
      MERCHANTABILITY, or FITNESS FOR A PARTICULAR PURPOSE. You are solely
      responsible for determining the appropriateness of using or
      redistributing the Work and assume any risks associated with Your
      exercise of permissions under this License.

   8. Limitation of Liability. In no event and under no legal theory,
      whether in tort (including negligence), contract, or otherwise,
      unless required by applicable law (such as deliberate and grossly
      negligent acts) or agreed to in writing, shall any Contributor be
      liable to You for damages, including any direct, indirect, special,
      incidental, or consequential damages of any character arising as a
      result of this License or out of the use or inability to use the
      Work (including but not limited to damages for loss of goodwill,
      work stoppage, computer failure or malfunction, or any and all
      other commercial damages or losses), even if such Contributor
      has been advised of the possibility of such damages.

   9. Accepting Warranty or Additional Liability. While redistributing
      the Work or Derivative Works thereof, You may choose to offer,
      and charge a fee for, acceptance of support, warranty, indemnity,
      or other liability obligations and/or rights consistent with this
      License. However, in accepting such obligations, You may act only
      on Your own behalf and on Your sole responsibility, not on behalf
      of any other Contributor, and only if You agree to indemnify,
      defend, and hold each Contributor harmless for any liability
      incurred by, or claims asserted against, such Contributor by reason
      of your accepting any such warranty or additional liability.

   END OF TERMS AND CONDITIONS

   APPENDIX: How to apply the Apache License to your work.

      To apply the Apache License to your document, just attach the following
      boilerplate notice, with the fields enclosed by brackets "{}" replaced
      with your own identifying information. (Don't include the brackets!)
      The text should be enclosed in the appropriate comment syntax for the
      file format. We also recommend that a file named NOTICE is distributed
      along with the Work; whatever the default distribution named methods
      of the Source Code work, such as a text file along with binary distributions.


   Copyright [yyyy] [name of author organization]

   Licensed under the Apache License, Version 2.0 (the "License");
   you may not use this file except in compliance with the License.
   You may obtain a copy of the License at:

       http://www.apache.org/licenses/LICENSE-2.0

   Unless required by applicable law or agreed to in writing, software
   distributed under the License is distributed on an "AS IS" BASIS,
   WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
   See the License for the specific language governing permissions and
   limitations under the License.

<a name="contact"></a>
## Contact

If you have any questions or suggestions, feel free to contact:

Mr. Awais Qureshi (MS-IT),
NUST SEECS,
aqureshi.dphd19seecs@seecs.edu.pk